                                            Automated text extraction

In [3]:
#libraries

from __future__ import unicode_literals, print_function
import cv2
import pytesseract
import matplotlib.pyplot as plt
import spacy
import plac
import random
from pathlib import Path
from tqdm import tqdm
import fileinput
from pdf2image import convert_from_path 
import os 
from PIL import Image  
import csv
import sys 
from tkinter import * 
from tkinter.ttk import *
import pickle
from tkinter.filedialog import askopenfile 
import fileinput
import docx
from docx import Document
from docx2pdf import convert
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import timedelta
from datetime import datetime

In [48]:
#function to perform ocr

def ocr(path,nop=0):
    PDF_file = path
    pages = convert_from_path(PDF_file, 500) 
    image_counter = 1
    for page in pages: 
        filename = "page_"+str(image_counter)+".jpg"
        page.save(filename, 'JPEG') 
        image_counter = image_counter + 1
    if nop!=0:
        filelimit = nop
    else:
        filelimit = image_counter-1
    outfile = "out_text.txt"
    f = open(outfile, "a") 
    output_text = ""
    for i in range(1, filelimit + 1): 
        filename = "page_"+str(i)+".jpg"
#         text = str(((pytesseract.image_to_string(Image.open(filename))))) 
        text = str(((pytesseract.image_to_string(filename)))) 
        text = text.replace('-\n', '')     
        output_text = output_text+text
        f.write(text) 
    f.close() 
    return output_text

In [240]:
#ocr function call

output_text = ocr(r"C:\Users\Sivaraman\Desktop\Py_projects\project_ocr\extract_2.pdf")

In [241]:
# saving text 

with open("textPickle","ab") as f:
    pickle.dump(output_text,f)
f.close()

In [4]:
#loading text

with open("textPickle","rb") as f:
    op_text = pickle.load(f)
f.close()

In [5]:
#printing text

op_txt_stp = op_text.rstrip()
op_txt_stp = op_txt_stp.rstrip()
op_txt_stp = op_txt_stp.lstrip()
print(op_txt_stp)

SAM tee PAID
on HHO bok
HOB CASH OFFICE

TEMPORARY OCCUPATION LICENCE FOR
AUTOMATED TELLER MACHINES

This Licence Agreement is made on the hig day of spre mM ber - OS
between the Housing and Development Board of HDB Hub, 480 Lorong 6 Toa Payoh Singapore
310480 (“HDB”) and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore

018982 (“Licensee”).

WHEREAS HDB has agreed to grant to the Licensee a Licence to use and occupy the space at
Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to
the terms and conditions of this Agreement.

It is hereby agreed as follows:

1 In consideration of the payment by the Licensee a sum of $800.00 per month (exclusive
of Goods and Services Tax) or such other sum as the HDB may notify in writing, HDB
hereby authorises the Licensee to use and occupy the Premises for the sole purpose of
setting up and maintaining an Automated Teller Machine (ATM) for a period of 3 years
commencing from 01/12/2018, subject

In [6]:
#function to remove line ends in sentences

def space_text(op_txt_stp):
    new_text = ""
    for i in range(len(op_txt_stp)):
        if op_txt_stp[i] == '\n' and op_txt_stp[i+1] != '\n':
                new_text = new_text + " "
        else:
            new_text = new_text + op_txt_stp[i]
    return new_text
new_text = space_text(op_txt_stp)

In [162]:
#applying spacy ner to text

nlp = spacy.load('en_core_web_sm') 
text = op_txt_stp
doc = nlp(text)
name = []
nric = []
for ent in doc.ents:
    if ent.label_=="ORG" or ent.label_ == "GPE" or ent.label_== "LOC":
        print(ent.text , ent.label_)

STAMP FEE PAID ORG
the Housing and Development Board of HDB Hub ORG
Singapore GPE
DBS Bank Ltd. ORG
Marina Boulevard LOC
SINGAPORE GPE
Licensee GPE
Goods and Services Tax ORG
Premises LOC
TOL ORG
TOL ORG
Goods and Services Tax ORG
Town Council ORG
the Town Council Act ORG
the Housing and Development Act ORG
Premises LOC
Premises LOC


In [225]:
#training data with non-zero entities

TRAIN_DATA = [('WHEREAS 2C2P has agreed to grant to the Licensee a Licence to use and occupy the space at Pyramid 87 Tuas Avenue 1 Singapore 555294 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 114, 'BA'), (115, 124, 'BC'), (125, 131, 'BP')]}), ('WHEREAS Aetos Security Management has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 184 Toa Payoh Central #03-366 S() Singapore 636657 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 148, 'BA'), (149, 158, 'BC'), (159, 165, 'BP')]}), ('WHEREAS AIBI International has agreed to grant to the Licensee a Licence to use and occupy the space at LOWER KENT RIDGE ROAD Singapore 722539 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS Antlabs has agreed to grant to the Licensee a Licence to use and occupy the space at Wing on Life Gardens 335 Bukit Timah Road #13-03 Singapore 588156 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 141, 'BA'), (142, 151, 'BC'), (152, 158, 'BP')]}), ('WHEREAS Aspial Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at 32 Sungei Kadut Way HUA KOK INDUSTRIAL BUILDING Singapore 717318 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 151, 'BA'), (152, 161, 'BC'), (162, 168, 'BP')]}), ('WHEREAS Ayam Brand has agreed to grant to the Licensee a Licence to use and occupy the space at 222 222 Tagore Lane 02-11 Tg Building Singapore 305242 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS Bee Cheng Hiang has agreed to grant to the Licensee a Licence to use and occupy the space at 72 Bendemeer Rd #07-03 Luzerne S() Singapore 728742 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 135, 'BA'), (136, 145, 'BC'), (146, 152, 'BP')]}), ('WHEREAS Boustead has agreed to grant to the Licensee a Licence to use and occupy the space at 146 Pandan Loop Blk 10 Singapore 276403 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 116, 'BA'), (117, 126, 'BC'), (127, 133, 'BP')]}), ('WHEREAS BreadTalk has agreed to grant to the Licensee a Licence to use and occupy the space at 2 Kallang Avenue 08-01/03 Kallang Bahru Complex Singapore 977633 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 142, 'BA'), (143, 152, 'BC'), (153, 159, 'BP')]}), ('WHEREAS Broadcom Limited has agreed to grant to the Licensee a Licence to use and occupy the space at Unit No.285, Kaki Bukit Singapore 627979 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS CapitaLand has agreed to grant to the Licensee a Licence to use and occupy the space at Cairnhill Shop Houses 56 Cairnhill Road Singapore 899103 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 135, 'BA'), (136, 145, 'BC'), (146, 152, 'BP')]}), ('WHEREAS Carousell has agreed to grant to the Licensee a Licence to use and occupy the space at Duchess Crest 58 Duchess Avenue #02-08 Singapore 837790 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS Certis CISCO has agreed to grant to the Licensee a Licence to use and occupy the space at CHANGI AIRPORT CARGO COMPLEX PO BOX 924 Singapore 532624 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 20, 'BN'), (98, 137, 'BA'), (138, 147, 'BC'), (148, 154, 'BP')]}), ('WHEREAS Charles & Keith has agreed to grant to the Licensee a Licence to use and occupy the space at 391A ORCHARD RD 12-06/07/08 NGEE ANN CITY TOWER A Singapore 952323 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 150, 'BA'), (151, 160, 'BC'), (161, 167, 'BP')]}), ('WHEREAS China Aviation Oil has agreed to grant to the Licensee a Licence to use and occupy the space at 4 Sago Lane 02-111 Singapore 424787 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 122, 'BA'), (123, 132, 'BC'), (133, 139, 'BP')]}), ('WHEREAS ComfortDelGro has agreed to grant to the Licensee a Licence to use and occupy the space at Lucky Plaza 304 Orchard Road #03-43 Singapore 415540 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 21, 'BN'), (99, 134, 'BA'), (135, 144, 'BC'), (145, 151, 'BP')]}), ('WHEREAS Creative Technology has agreed to grant to the Licensee a Licence to use and occupy the space at 748 Yishun Street 72 01-212 Singapore 246659 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 27, 'BN'), (105, 132, 'BA'), (133, 142, 'BC'), (143, 149, 'BP')]}), ('WHEREAS dnata has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 602 AMK Ave 5 ,02-2655 Singapore 867869 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 117, 'BA'), (118, 127, 'BC'), (128, 134, 'BP')]}), ('WHEREAS Far East Orchard has agreed to grant to the Licensee a Licence to use and occupy the space at 448 Hougang Avenue 10 08-531 Singapore 420103 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 130, 'BA'), (131, 140, 'BC'), (141, 147, 'BP')]}), ('WHEREAS Far East Organization has agreed to grant to the Licensee a Licence to use and occupy the space at 100 High Street #06-01 The Treasury Singapore 682167 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 29, 'BN'), (107, 142, 'BA'), (143, 152, 'BC'), (153, 159, 'BP')]}), ('WHEREAS FilmTack has agreed to grant to the Licensee a Licence to use and occupy the space at 10-12 SCOTTS ROAD, #02 Singapore 655576 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 116, 'BA'), (117, 126, 'BC'), (127, 133, 'BP')]}), ('WHEREAS Flextronics has agreed to grant to the Licensee a Licence to use and occupy the space at 175A Bencoolen Street #05-11 Office Tower Singapore 805448 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 19, 'BN'), (97, 138, 'BA'), (139, 148, 'BC'), (149, 155, 'BP')]}), ('WHEREAS Fraser and Neave has agreed to grant to the Licensee a Licence to use and occupy the space at 219 Henderson Road Singapore 649728 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 120, 'BA'), (121, 130, 'BC'), (131, 137, 'BP')]}), ('WHEREAS Garena has agreed to grant to the Licensee a Licence to use and occupy the space at 361 Ubi Road 3 #04-01 Singapore 136891 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}), ('WHEREAS Genting has agreed to grant to the Licensee a Licence to use and occupy the space at 54 Genting Lane #01-04 Ruby Land Complex Blk 2 S() Singapore 569207 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 143, 'BA'), (144, 153, 'BC'), (154, 160, 'BP')]}), ('WHEREAS GlobalRoam has agreed to grant to the Licensee a Licence to use and occupy the space at 331 NORTH BRIDGE ROAD 14-06 Singapore 210498 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 123, 'BA'), (124, 133, 'BC'), (134, 140, 'BP')]}), ('WHEREAS Golden Agri-Resources has agreed to grant to the Licensee a Licence to use and occupy the space at 1803 Geylang Bahru 01-01 Singapore 296457 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 29, 'BN'), (107, 131, 'BA'), (132, 141, 'BC'), (142, 148, 'BP')]}), ('WHEREAS Grab has agreed to grant to the Licensee a Licence to use and occupy the space at 154 West Coast Road 01-115 Ginza Plaza Singapore 919784 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 128, 'BA'), (129, 138, 'BC'), (139, 145, 'BP')]}), ('WHEREAS Great Eastern Life has agreed to grant to the Licensee a Licence to use and occupy the space at Hup Lian Huat Engineering 38 Defu Lane 9 Singapore 223316 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 144, 'BA'), (145, 154, 'BC'), (155, 161, 'BP')]}), ('WHEREAS Hyflux has agreed to grant to the Licensee a Licence to use and occupy the space at Far East Shopping Centre 545 Orchard Road #04-14 Singapore 892492 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 140, 'BA'), (141, 150, 'BC'), (151, 157, 'BP')]}), ('WHEREAS Jetstar Asia Airways has agreed to grant to the Licensee a Licence to use and occupy the space at 21 Senoko South Road Woodlands East Singapore 965946 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 28, 'BN'), (106, 141, 'BA'), (142, 151, 'BC'), (152, 158, 'BP')]}), ('WHEREAS JTC Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at International Building 360 Orchard Road #12-04 Singapore 305386 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 147, 'BA'), (148, 157, 'BC'), (158, 164, 'BP')]}), ('WHEREAS Jurong Port has agreed to grant to the Licensee a Licence to use and occupy the space at Ion Orchard, 2 Orchard Turn, Singapore 763791 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 19, 'BN'), (97, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS Keppel Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at 25 Lorong Kilat #02-03 Singapore 343160 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 126, 'BA'), (127, 136, 'BC'), (137, 143, 'BP')]}), ('WHEREAS Koufu has agreed to grant to the Licensee a Licence to use and occupy the space at 20 Ang Mo Kio Industrial Park 2A 02-02 Singapore 693818 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 129, 'BA'), (130, 139, 'BC'), (140, 146, 'BP')]}), ('WHEREAS M1 Limited has agreed to grant to the Licensee a Licence to use and occupy the space at Paramount Shopping Centre 30 East Coast Road #02-08 Singapore 147305 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 147, 'BA'), (148, 157, 'BC'), (158, 164, 'BP')]}), ('WHEREAS Mediacorp has agreed to grant to the Licensee a Licence to use and occupy the space at 1 Finlayson Green #13-00 Singapore 425984 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 119, 'BA'), (120, 129, 'BC'), (130, 136, 'BP')]}), ('WHEREAS MyRepublic has agreed to grant to the Licensee a Licence to use and occupy the space at 20 Tuas Avenue 8 Singapore 435720 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 112, 'BA'), (113, 122, 'BC'), (123, 129, 'BP')]}), ('WHEREAS Neptune Orient Lines has agreed to grant to the Licensee a Licence to use and occupy the space at 57 UBI AVENUE 1, #05-14 Singapore 778909 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 28, 'BN'), (106, 129, 'BA'), (130, 139, 'BC'), (140, 146, 'BP')]}), ('WHEREAS NETS has agreed to grant to the Licensee a Licence to use and occupy the space at 501 Bukit Timah Road #01-05B Singapore 620727 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 118, 'BA'), (119, 128, 'BC'), (129, 135, 'BP')]}), ('WHEREAS NTUC FairPrice has agreed to grant to the Licensee a Licence to use and occupy the space at HDB Jurong 351 Jurong East Street 31 #01-81 Singapore 594529 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 22, 'BN'), (100, 143, 'BA'), (144, 153, 'BC'), (154, 160, 'BP')]}), ('WHEREAS OCBC Bank has agreed to grant to the Licensee a Licence to use and occupy the space at 32 Defu Lane 10 04-34 Singapore 915498 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 116, 'BA'), (117, 126, 'BC'), (127, 133, 'BP')]}), ('WHEREAS Osim International has agreed to grant to the Licensee a Licence to use and occupy the space at Haw Par Technocentre #01-04, 401 Commonwealth Drive Singapore 293654 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 155, 'BA'), (156, 165, 'BC'), (166, 172, 'BP')]}), ('WHEREAS Pacific Century Regional Developments Limited has agreed to grant to the Licensee a Licence to use and occupy the space at 8 Jalan Kilang Timor #02-01 KEWALRAM HOUSE Singapore 862444 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 53, 'BN'), (131, 173, 'BA'), (174, 183, 'BC'), (184, 190, 'BP')]}), ('WHEREAS Popular Holdings has agreed to grant to the Licensee a Licence to use and occupy the space at 166 Bukit Merah Central 01-3531 Singapore 102127 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS POSB Bank has agreed to grant to the Licensee a Licence to use and occupy the space at Changi Airport Terminal 1, 80 Airport Boulevard, Singapore 954993 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 143, 'BA'), (144, 153, 'BC'), (154, 160, 'BP')]}), ('WHEREAS PSA International has agreed to grant to the Licensee a Licence to use and occupy the space at 27 Jln Melor Singapore 877704 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 25, 'BN'), (103, 115, 'BA'), (116, 125, 'BC'), (126, 132, 'BP')]}), ('WHEREAS Quest Global has agreed to grant to the Licensee a Licence to use and occupy the space at 1002 Jln Bt Merah #07-11 Redhill Ind Est S() Singapore 127915 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 20, 'BN'), (98, 142, 'BA'), (143, 152, 'BC'), (153, 159, 'BP')]}), ('WHEREAS Renewable Energy Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at 163 Tanglin Road 03-19 Tanglin Mall Singapore 255911 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 36, 'BN'), (114, 149, 'BA'), (150, 159, 'BC'), (160, 166, 'BP')]}), ('WHEREAS Robinsons & Co. has agreed to grant to the Licensee a Licence to use and occupy the space at 354 Clementi Ave 2 #01-253 Singapore 657574 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS SATS Ltd has agreed to grant to the Licensee a Licence to use and occupy the space at 9010 Tampines St 93 #04-109 Singapore 118699 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 121, 'BA'), (122, 131, 'BC'), (132, 138, 'BP')]}), ('WHEREAS SBS Transit has agreed to grant to the Licensee a Licence to use and occupy the space at Bedok Clinic & Surgery 17 Bedok South Road #01-581 Singapore 333904 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 19, 'BN'), (97, 147, 'BA'), (148, 157, 'BC'), (158, 164, 'BP')]}), ('WHEREAS Scoot has agreed to grant to the Licensee a Licence to use and occupy the space at 1 Coleman Street 02-38 Adelphi The Singapore 375835 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS SembCorp Marine has agreed to grant to the Licensee a Licence to use and occupy the space at 230 Orchard Road #10-230 Faber House Singapore 659453 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 137, 'BA'), (138, 147, 'BC'), (148, 154, 'BP')]}), ('WHEREAS SGAG has agreed to grant to the Licensee a Licence to use and occupy the space at 583 ORCHARD ROAD, #14-02 Singapore 841193 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 114, 'BA'), (115, 124, 'BC'), (125, 131, 'BP')]}), ('WHEREAS Sheng Siong has agreed to grant to the Licensee a Licence to use and occupy the space at The Paterson Edge 26 Paterson Road #01-00 Singapore 947834 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 19, 'BN'), (97, 138, 'BA'), (139, 148, 'BC'), (149, 155, 'BP')]}), ('WHEREAS Shopee has agreed to grant to the Licensee a Licence to use and occupy the space at 10 Admiralty St #05-25 North Link Bldg S() Singapore 806333 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 134, 'BA'), (135, 144, 'BC'), (145, 151, 'BP')]}), ('WHEREAS SIA Engineering Company has agreed to grant to the Licensee a Licence to use and occupy the space at 10 Anson Road 11-10 International Plaza Singapore 933044 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 31, 'BN'), (109, 148, 'BA'), (149, 158, 'BC'), (159, 165, 'BP')]}), ('WHEREAS SilkAir has agreed to grant to the Licensee a Licence to use and occupy the space at BLK 231 BISHAN STREET 23#B1-05 Singapore 262052 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 123, 'BA'), (124, 133, 'BC'), (134, 140, 'BP')]}), ('WHEREAS Airlines has agreed to grant to the Licensee a Licence to use and occupy the space at BLK 682 HOUGANG AVENUE 4#01-322 Singapore 922699 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS Airlines Cargo has agreed to grant to the Licensee a Licence to use and occupy the space at 143 Cecil Street 18-00 Gb Building Singapore 454787 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 22, 'BN'), (100, 134, 'BA'), (135, 144, 'BC'), (145, 151, 'BP')]}), ('WHEREAS Exchange has agreed to grant to the Licensee a Licence to use and occupy the space at 1D Yong Siak Street Singapore 602875 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}), ('WHEREAS Petroleum Company Limited has agreed to grant to the Licensee a Licence to use and occupy the space at 21, Kings Polemeller Plaza, Smeily Street Singapore 826024 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 152, 'BA'), (153, 162, 'BC'), (163, 169, 'BP')]}), ('WHEREAS Post has agreed to grant to the Licensee a Licence to use and occupy the space at 09-13 ALEXANDRA DISTRIPARK BLK 4 Singapore 453838 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 122, 'BA'), (123, 132, 'BC'), (133, 139, 'BP')]}), ('WHEREAS Press Holdings has agreed to grant to the Licensee a Licence to use and occupy the space at 10 Shenton Way 11-03 M A S Building Singapore 745153 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 22, 'BN'), (100, 135, 'BA'), (136, 145, 'BC'), (146, 152, 'BP')]}), ('WHEREAS Singtel has agreed to grant to the Licensee a Licence to use and occupy the space at 210 Middle Road #11-06 IOI Plaza Singapore 305480 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 125, 'BA'), (126, 135, 'BC'), (136, 142, 'BP')]}), ('WHEREAS SMRT Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at 3 Pasir Panjang Rd Alexandra Distripark Singapore 431905 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 141, 'BA'), (142, 151, 'BC'), (152, 158, 'BP')]}), ('WHEREAS SP Group has agreed to grant to the Licensee a Licence to use and occupy the space at HDB Pasir Ris 419 Pasir Ris Drive 6 #08-275 Singapore 687419 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 137, 'BA'), (138, 147, 'BC'), (148, 154, 'BP')]}), ('WHEREAS ST Engineering has agreed to grant to the Licensee a Licence to use and occupy the space at 85 Science Park Drive, ,01-01&01-02 The Cavendish Singapore 412539 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 22, 'BN'), (100, 149, 'BA'), (150, 159, 'BC'), (160, 166, 'BP')]}), ('WHEREAS Standard Chartered has agreed to grant to the Licensee a Licence to use and occupy the space at 115 AIRPORT CARGO ROAD 06-18 CARGO AGENTS BUILDING C Singapore 470174 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 26, 'BN'), (104, 156, 'BA'), (157, 166, 'BC'), (167, 173, 'BP')]}), ('WHEREAS StarHub has agreed to grant to the Licensee a Licence to use and occupy the space at 72 Bendemeer Road #03-31 Luzerne (Formerly Known As Hiap Huat House) Singapore 919746 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 161, 'BA'), (162, 171, 'BC'), (172, 178, 'BP')]}), ('WHEREAS Systems on Silicon Manufacturing has agreed to grant to the Licensee a Licence to use and occupy the space at 1 Scotts Road #03-04 Shaw Centre Singapore 621306 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 40, 'BN'), (118, 150, 'BA'), (151, 160, 'BC'), (161, 167, 'BP')]}), ('WHEREAS Tangs has agreed to grant to the Licensee a Licence to use and occupy the space at 25 INTERNATIONAL BUSINESS PARK 03-20 GERMAN CENTRE Singapore 538582 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 141, 'BA'), (142, 151, 'BC'), (152, 158, 'BP')]}), ('WHEREAS Tee Yih Jia has agreed to grant to the Licensee a Licence to use and occupy the space at 49 Jalan Pemimpin #05-04 Aps Industrial Building Singapore 160535 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 19, 'BN'), (97, 145, 'BA'), (146, 155, 'BC'), (156, 162, 'BP')]}), ('WHEREAS Temasek Holdings has agreed to grant to the Licensee a Licence to use and occupy the space at 201 Keppel Road Blk 6 Singapore 693540 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 123, 'BA'), (124, 133, 'BC'), (134, 140, 'BP')]}), ('WHEREAS Thakral Corporation has agreed to grant to the Licensee a Licence to use and occupy the space at 7 Tuas Ave 2 Singapore 494280 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 27, 'BN'), (105, 117, 'BA'), (118, 127, 'BC'), (128, 134, 'BP')]}), ('WHEREAS Tiger Airways Holdings has agreed to grant to the Licensee a Licence to use and occupy the space at 9 Lorong 21A Geylang #03-00 Singapore 550090 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 30, 'BN'), (108, 135, 'BA'), (136, 145, 'BC'), (146, 152, 'BP')]}), ('WHEREAS Twelve Cupcakes has agreed to grant to the Licensee a Licence to use and occupy the space at 2 Toh Tuck Link #03-00 S() Singapore 952049 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 23, 'BN'), (101, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS Vertex Venture Holdings has agreed to grant to the Licensee a Licence to use and occupy the space at 18 Sungei Tengah Road Singapore 799837 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 31, 'BN'), (109, 130, 'BA'), (131, 140, 'BC'), (141, 147, 'BP')]}), ('WHEREAS Wilmar has agreed to grant to the Licensee a Licence to use and occupy the space at 51 Thomson Road #181B Goldhill Centre Singapore 129211 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 129, 'BA'), (130, 139, 'BC'), (140, 146, 'BP')]}), ('WHEREAS Ya Kun Kaya Toast has agreed to grant to the Licensee a Licence to use and occupy the space at 504 JURONG WEST STREET 51 02-213 Singapore 252215 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 25, 'BN'), (103, 135, 'BA'), (136, 145, 'BC'), (146, 152, 'BP')]}), ('WHEREAS Yeo Hiap Seng has agreed to grant to the Licensee a Licence to use and occupy the space at 201E TAMPINES STREET 23 01-72 Singapore 664888 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 21, 'BN'), (99, 128, 'BA'), (129, 138, 'BC'), (139, 145, 'BP')]}), ('WHEREAS ACCOUNTING AND CORPORATE REGULATORY AUTHORITY (ACRA) has agreed to grant to the Licensee a Licence to use and occupy the space at Guan Hua Warehouse Building 85 Genting Lane #02-02 Singapore 420087 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 60, 'BN'), (138, 188, 'BA'), (189, 198, 'BC'), (199, 205, 'BP')]}), ('WHEREAS AGENCY FOR SCIENCE, TECHNOLOGY AND RESEARCH (A*STAR) has agreed to grant to the Licensee a Licence to use and occupy the space at 1 FISHERY PORT ROAD Singapore 768852 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 60, 'BN'), (138, 157, 'BA'), (158, 167, 'BC'), (168, 174, 'BP')]}), ('WHEREAS BOARD OF ARCHITECTS (BOA) has agreed to grant to the Licensee a Licence to use and occupy the space at 331 North Bridge Road ,04-06 Odeon Towers Singapore 699894 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 152, 'BA'), (153, 162, 'BC'), (163, 169, 'BP')]}), ('WHEREAS BUILDING AND CONSTRUCTION AUTHORITY (BCA) has agreed to grant to the Licensee a Licence to use and occupy the space at 116 CHANGI ROAD, #02-05 Singapore 655392 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 49, 'BN'), (127, 150, 'BA'), (151, 160, 'BC'), (161, 167, 'BP')]}), ('WHEREAS CASINO REGULATORY AUTHORITY OF (CRA) has agreed to grant to the Licensee a Licence to use and occupy the space at 6 BATTERY RD 11-07 Singapore 842796 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 44, 'BN'), (122, 140, 'BA'), (141, 150, 'BC'), (151, 157, 'BP')]}), ('WHEREAS CENTRAL PROVIDENT FUND BOARD (CPFB) has agreed to grant to the Licensee a Licence to use and occupy the space at 160 Paya Lebar Road 07-08 Orion Industrial Building Singapore 216383 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 43, 'BN'), (121, 172, 'BA'), (173, 182, 'BC'), (183, 189, 'BP')]}), ('WHEREAS CIVIL AVIATION AUTHORITY OF (CAAS) has agreed to grant to the Licensee a Licence to use and occupy the space at 304 Orchard Rd #03-62 Singapore 161192 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 42, 'BN'), (120, 141, 'BA'), (142, 151, 'BC'), (152, 158, 'BP')]}), ('WHEREAS CIVIL SERVICE COLLEGE (CSC) has agreed to grant to the Licensee a Licence to use and occupy the space at 11 Berwick Dr S() Singapore 289008 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 35, 'BN'), (113, 130, 'BA'), (131, 140, 'BC'), (141, 147, 'BP')]}), ('WHEREAS COMPETITION AND CONSUMER COMMISSION OF (CCCS) has agreed to grant to the Licensee a Licence to use and occupy the space at No.2 Bukit Merah Central, #10-00 Spring Building, 5 Singapore 980783 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 53, 'BN'), (131, 182, 'BA'), (183, 192, 'BC'), (193, 199, 'BP')]}), ('WHEREAS COUNCIL FOR ESTATE AGENCIES (CEA) has agreed to grant to the Licensee a Licence to use and occupy the space at 3 ROCHOR ROAD 02-628 ROCHOR CENTRE Singapore 281201 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 41, 'BN'), (119, 153, 'BA'), (154, 163, 'BC'), (164, 170, 'BP')]}), ('WHEREAS DEFENCE SCIENCE AND TECHNOLOGY AGENCY (DSTA) has agreed to grant to the Licensee a Licence to use and occupy the space at 2 Haig Rd #04-517 Singapore 718374 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 52, 'BN'), (130, 147, 'BA'), (148, 157, 'BC'), (158, 164, 'BP')]}), ('WHEREAS ECONOMIC DEVELOPMENT BOARD (EDB) has agreed to grant to the Licensee a Licence to use and occupy the space at 400 ORCHARD ROAD, #03-11 Singapore 323532 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 40, 'BN'), (118, 142, 'BA'), (143, 152, 'BC'), (153, 159, 'BP')]}), ('WHEREAS ENERGY MARKET AUTHORITY (EMA) has agreed to grant to the Licensee a Licence to use and occupy the space at 3 Temasek Boulevard B1-024 Suntec City Mall Singapore 212321 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 37, 'BN'), (115, 158, 'BA'), (159, 168, 'BC'), (169, 175, 'BP')]}), ('WHEREAS ENTERPRISE (ESG) has agreed to grant to the Licensee a Licence to use and occupy the space at 446 Bright Hill Drive 02-111 Singapore 861237 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 24, 'BN'), (102, 130, 'BA'), (131, 140, 'BC'), (141, 147, 'BP')]}), ('WHEREAS GOVERNMENT TECHNOLOGY AGENCY (GOVTECH) has agreed to grant to the Licensee a Licence to use and occupy the space at 111 North Bridge Road #27-01 Peninsula Plaza Singapore 218131 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 46, 'BN'), (124, 168, 'BA'), (169, 178, 'BC'), (179, 185, 'BP')]}), ('WHEREAS HEALTH PROMOTION BOARD (HPB) has agreed to grant to the Licensee a Licence to use and occupy the space at 2 Bayfront Avenue, #B1-60 Singapore 310079 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 36, 'BN'), (114, 139, 'BA'), (140, 149, 'BC'), (150, 156, 'BP')]}), ('WHEREAS HEALTH SCIENCES AUTHORITY (HSA) has agreed to grant to the Licensee a Licence to use and occupy the space at 767 Geylang Road Singapore 454106 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 39, 'BN'), (117, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS HOME TEAM SCIENCE AND TECHNOLOGY AGENCY (HTX) has agreed to grant to the Licensee a Licence to use and occupy the space at 8 Loyang Drive Singapore 623578 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 53, 'BN'), (131, 145, 'BA'), (146, 155, 'BC'), (156, 162, 'BP')]}), ('WHEREAS HOTELS LICENSING BOARD (HLB) has agreed to grant to the Licensee a Licence to use and occupy the space at No. 1 Marine Parade Central, 12-07 Parway Centre Singapore 927292 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 36, 'BN'), (114, 162, 'BA'), (163, 172, 'BC'), (173, 179, 'BP')]}), ('WHEREAS HOUSING & DEVELOPMENT BOARD (HDB) has agreed to grant to the Licensee a Licence to use and occupy the space at 30 Bukit Batok Street 31 #04-01 Hong Kah North Community Centre Singapore 742417 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 41, 'BN'), (119, 182, 'BA'), (183, 192, 'BC'), (193, 199, 'BP')]}), ('WHEREAS INFO-COMMUNICATIONS MEDIA DEVELOPMENT AUTHORITY (IMDA) has agreed to grant to the Licensee a Licence to use and occupy the space at 6A Napier Road #02-38 Gleneagles Hospital Singapore 182472 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 62, 'BN'), (140, 181, 'BA'), (182, 191, 'BC'), (192, 198, 'BP')]}), ('WHEREAS INLAND REVENUE AUTHORITY OF (IRAS) has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 20 Ang Mo Kio Industrial Park 2A ,05-11 AMK Techlink Singapore 226885 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 42, 'BN'), (120, 176, 'BA'), (177, 186, 'BC'), (187, 193, 'BP')]}), ('WHEREAS INSTITUTE OF TECHNICAL EDUCATION (ITE) has agreed to grant to the Licensee a Licence to use and occupy the space at 27 Tuas Avenue 13 01-22 Singapore 893447 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 46, 'BN'), (124, 147, 'BA'), (148, 157, 'BC'), (158, 164, 'BP')]}), ('WHEREAS INTELLECTUAL PROPERTY OFFICE OF (IPOS) has agreed to grant to the Licensee a Licence to use and occupy the space at Lorong 1 Toa Payoh, Oleander Towers Singapore 629739 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 46, 'BN'), (124, 159, 'BA'), (160, 169, 'BC'), (170, 176, 'BP')]}), ('WHEREAS ISEAS-YUSOF ISHAK INSTITUTE (ISEAS) has agreed to grant to the Licensee a Licence to use and occupy the space at 5 SHENTON WAY, #08-02 Singapore 915879 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 43, 'BN'), (121, 142, 'BA'), (143, 152, 'BC'), (153, 159, 'BP')]}), ('WHEREAS JTC CORPORATION (JTC) has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 9 #01-12 Dempsey Road, Dempsey Hill Singapore 351481 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 29, 'BN'), (107, 146, 'BA'), (147, 156, 'BC'), (157, 163, 'BP')]}), ('WHEREAS LAND SURVEYORS BOARD (LSB) has agreed to grant to the Licensee a Licence to use and occupy the space at 9B Hoe Chiang Road Singapore 548212 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 34, 'BN'), (112, 130, 'BA'), (131, 140, 'BC'), (141, 147, 'BP')]}), ('WHEREAS LAND TRANSPORT AUTHORITY (LTA) has agreed to grant to the Licensee a Licence to use and occupy the space at 42 Woodlands Terrace Singapore 614655 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 38, 'BN'), (116, 136, 'BA'), (137, 146, 'BC'), (147, 153, 'BP')]}), ('WHEREAS MAJLIS UGAMA ISLAM SINGAPURA (MUIS) has agreed to grant to the Licensee a Licence to use and occupy the space at 304 Orchard Road 01-53 Lucky Plaza Singapore 478662 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 43, 'BN'), (121, 155, 'BA'), (156, 165, 'BC'), (166, 172, 'BP')]}), ('WHEREAS MARITIME AND PORT AUTHORITY OF (MPA) has agreed to grant to the Licensee a Licence to use and occupy the space at BLK 105 ANG MO KIO AVENUE 4, #06-194 Singapore 728841 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 44, 'BN'), (122, 158, 'BA'), (159, 168, 'BC'), (169, 175, 'BP')]}), ('WHEREAS MONETARY AUTHORITY OF (MAS) has agreed to grant to the Licensee a Licence to use and occupy the space at 30 HONGKONG STREET Singapore 743828 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 35, 'BN'), (113, 131, 'BA'), (132, 141, 'BC'), (142, 148, 'BP')]}), ('WHEREAS NANYANG POLYTECHNIC (NYP) has agreed to grant to the Licensee a Licence to use and occupy the space at 4 Kwong Min Road Singapore 559336 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS NATIONAL ARTS COUNCIL (NAC) has agreed to grant to the Licensee a Licence to use and occupy the space at LCK25 LIM CHU KANG LANE 3 Singapore 647406 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 35, 'BN'), (113, 138, 'BA'), (139, 148, 'BC'), (149, 155, 'BP')]}), ('WHEREAS NATIONAL COUNCIL OF SOCIAL SERVICE (NCSS) has agreed to grant to the Licensee a Licence to use and occupy the space at 360 Orchard Road #09-01 INTERNATIONAL BUILDING Singapore 838030 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 49, 'BN'), (127, 173, 'BA'), (174, 183, 'BC'), (184, 190, 'BP')]}), ('WHEREAS NATIONAL ENVIRONMENT AGENCY (NEA) has agreed to grant to the Licensee a Licence to use and occupy the space at 420 North Bridge Road #03-07 North Bridge Centre Singapore 769977 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 41, 'BN'), (119, 167, 'BA'), (168, 177, 'BC'), (178, 184, 'BP')]}), ('WHEREAS NATIONAL HERITAGE BOARD (NHB) has agreed to grant to the Licensee a Licence to use and occupy the space at 364 JOO CHIAT ROAD Singapore 239679 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 37, 'BN'), (115, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS NATIONAL LIBRARY BOARD (NLB) has agreed to grant to the Licensee a Licence to use and occupy the space at 89 Circuit Rd #01-941A Singapore 991833 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 36, 'BN'), (114, 136, 'BA'), (137, 146, 'BC'), (147, 153, 'BP')]}), ('WHEREAS NATIONAL PARKS BOARD (NPARKS) has agreed to grant to the Licensee a Licence to use and occupy the space at lot lck 34, 31 lim chu kang lane 6c, Singapore 132761 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 37, 'BN'), (115, 151, 'BA'), (152, 161, 'BC'), (162, 168, 'BP')]}), ('WHEREAS NGEE ANN POLYTECHNIC (NP) has agreed to grant to the Licensee a Licence to use and occupy the space at 9, Jalan Pinang, Singapore 158355 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS PEOPLE\'S ASSOCIATION (PA) has agreed to grant to the Licensee a Licence to use and occupy the space at 605B Macpherson Road #04-13 Citimac Industrial Complex B Singapore 661924 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 33, 'BN'), (111, 167, 'BA'), (168, 177, 'BC'), (178, 184, 'BP')]}), ('WHEREAS PROFESSIONAL ENGINEERS BOARD, (PEB) has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 244 Jurong East St 24 Singapore 125016 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 43, 'BN'), (121, 146, 'BA'), (147, 156, 'BC'), (157, 163, 'BP')]}), ('WHEREAS PUB has agreed to grant to the Licensee a Licence to use and occupy the space at Tampines St 41, Tampines Singapore 850343 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}), ('WHEREAS PTC has agreed to grant to the Licensee a Licence to use and occupy the space at 60 COLLYER QUAY 32 Singapore 619186 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 107, 'BA'), (108, 117, 'BC'), (118, 124, 'BP')]}), ('WHEREAS RP has agreed to grant to the Licensee a Licence to use and occupy the space at BLK 119 ALJUNIED AVENUE 2, #01-K4 Singapore 814692 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 10, 'BN'), (88, 121, 'BA'), (122, 131, 'BC'), (132, 138, 'BP')]}), ('WHEREAS SCB has agreed to grant to the Licensee a Licence to use and occupy the space at 11 Haji Lane S() Singapore 269539 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 105, 'BA'), (106, 115, 'BC'), (116, 122, 'BP')]}), ('WHEREAS SDC has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 128 Kim Tian Road# 16-119 Singapore 796865 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 118, 'BA'), (119, 128, 'BC'), (129, 135, 'BP')]}), ('WHEREAS SAC has agreed to grant to the Licensee a Licence to use and occupy the space at 9 Kaki Bukit Rd 2 #01-09 Singapore 391920 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}), ('WHEREAS SDC has agreed to grant to the Licensee a Licence to use and occupy the space at 275 Thomson Road 01-10 Novena Ville Singapore 512171 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 124, 'BA'), (125, 134, 'BC'), (135, 141, 'BP')]}), ('WHEREAS SEAB has agreed to grant to the Licensee a Licence to use and occupy the space at Suntec Tower Two 9 Temasek Boulevard #04-01 Singapore 546462 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS SFA has agreed to grant to the Licensee a Licence to use and occupy the space at 144 UPPER BUKIT TIMAH ROAD, #02-17 Singapore 575841 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 123, 'BA'), (124, 133, 'BC'), (134, 140, 'BP')]}), ('WHEREAS SLF has agreed to grant to the Licensee a Licence to use and occupy the space at #04-33V, IMM Building, No.2 Jurong East Street 21 Singapore 526435 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 138, 'BA'), (139, 148, 'BC'), (149, 155, 'BP')]}), ('WHEREAS SLA has agreed to grant to the Licensee a Licence to use and occupy the space at 49 TANNERY LANE 05-03 Singapore 448638 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 110, 'BA'), (111, 120, 'BC'), (121, 127, 'BP')]}), ('WHEREAS SMC has agreed to grant to the Licensee a Licence to use and occupy the space at 1 1 North Bridge Road 01-01 High St Centre Singapore 334252 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 131, 'BA'), (132, 141, 'BC'), (142, 148, 'BP')]}), ('WHEREAS SNB has agreed to grant to the Licensee a Licence to use and occupy the space at 290 Orchard Rd #09-07 Singapore 134328 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 110, 'BA'), (111, 120, 'BC'), (121, 127, 'BP')]}), ('WHEREAS SPC has agreed to grant to the Licensee a Licence to use and occupy the space at Fortune Centre, 190 Middle Road, #10-08 Singapore 609242 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 128, 'BA'), (129, 138, 'BC'), (139, 145, 'BP')]}), ('WHEREAS SP has agreed to grant to the Licensee a Licence to use and occupy the space at 51 Lorong 3 Geylang 05-01 Singapore 728670 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 10, 'BN'), (88, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}), ('WHEREAS STB has agreed to grant to the Licensee a Licence to use and occupy the space at 123 McNair Rd #10-07 Singapore 281874 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 109, 'BA'), (110, 119, 'BC'), (120, 126, 'BP')]}), ('WHEREAS SSG has agreed to grant to the Licensee a Licence to use and occupy the space at 20 Martin Road #08-01 SENG KEE BUILDING Singapore 535086 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 128, 'BA'), (129, 138, 'BC'), (139, 145, 'BP')]}), ('WHEREAS SportSG has agreed to grant to the Licensee a Licence to use and occupy the space at 252 North Bridge Road Singapore 149607 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 114, 'BA'), (115, 124, 'BC'), (125, 131, 'BP')]}), ('WHEREAS TP has agreed to grant to the Licensee a Licence to use and occupy the space at 101 UPPER CROSS ST B1-03 PEOPLE\'S PARK CENTRE Singapore 666832 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 10, 'BN'), (88, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS TOTE BOARD has agreed to grant to the Licensee a Licence to use and occupy the space at 140 Robinson Road, #03-05, Chow House Singapore 973023 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 18, 'BN'), (96, 133, 'BA'), (134, 143, 'BC'), (144, 150, 'BP')]}), ('WHEREAS TCMPB has agreed to grant to the Licensee a Licence to use and occupy the space at 32 Quality Road Singapore 224567 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 106, 'BA'), (107, 116, 'BC'), (117, 123, 'BP')]}), ('WHEREAS URA has agreed to grant to the Licensee a Licence to use and occupy the space at 5 Kaki Bukit Road 2 #01-10 & #02-10 CITY WAREHOUSE Singapore 852714 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 139, 'BA'), (140, 149, 'BC'), (150, 156, 'BP')]}), ('WHEREAS WSG has agreed to grant to the Licensee a Licence to use and occupy the space at 24 Raffles Place 19-04 Clifford Centre Singapore 320380 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS YRSG has agreed to grant to the Licensee a Licence to use and occupy the space at 17 Soon Lee Road Singapore 387438 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 106, 'BA'), (107, 116, 'BC'), (117, 123, 'BP')]}), ('WHEREAS Samsung Electronics has agreed to grant to the Licensee a Licence to use and occupy the space at 3007 Ubi Road 1 05-400 Singapore 770349 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 27, 'BN'), (105, 127, 'BA'), (128, 137, 'BC'), (138, 144, 'BP')]}), ('WHEREAS Foxconn has agreed to grant to the Licensee a Licence to use and occupy the space at 155 Kew Cres S() Singapore 941272 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 109, 'BA'), (110, 119, 'BC'), (120, 126, 'BP')]}), ('WHEREAS Alphabet has agreed to grant to the Licensee a Licence to use and occupy the space at 7 Wholesales Centre #01-235 Singapore 560593 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 121, 'BA'), (122, 131, 'BC'), (132, 138, 'BP')]}), ('WHEREAS Microsoft has agreed to grant to the Licensee a Licence to use and occupy the space at 391 ORCHARD ROAD, #03-51 Singapore 997937 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 119, 'BA'), (120, 129, 'BC'), (130, 136, 'BP')]}), ('WHEREAS Huawei has agreed to grant to the Licensee a Licence to use and occupy the space at 802 Bukit Batok West Avenue 5 01-01 Bukit Gombak MRT Station Singapore 223966 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 152, 'BA'), (153, 162, 'BC'), (163, 169, 'BP')]}), ('WHEREAS Dell Technologies has agreed to grant to the Licensee a Licence to use and occupy the space at Hitachi Tower 16 Collyer Quay #30-01 Singapore 543846 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 25, 'BN'), (103, 139, 'BA'), (140, 149, 'BC'), (150, 156, 'BP')]}), ('WHEREAS Hitachi has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 782, Pasir Ris St. 71 Singapore 805722 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 15, 'BN'), (93, 118, 'BA'), (119, 128, 'BC'), (129, 135, 'BP')]}), ('WHEREAS IBM has agreed to grant to the Licensee a Licence to use and occupy the space at 14 HOUGANG AVENUE 7 01-563 Singapore 919220 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 11, 'BN'), (89, 115, 'BA'), (116, 125, 'BC'), (126, 132, 'BP')]}), ('WHEREAS Sony has agreed to grant to the Licensee a Licence to use and occupy the space at HDB-Ang Mo Kio 114 Ang Mo Kio Avenue 4 #01-353 Singapore 466572 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 12, 'BN'), (90, 136, 'BA'), (137, 146, 'BC'), (147, 153, 'BP')]}), ('WHEREAS Panasonic has agreed to grant to the Licensee a Licence to use and occupy the space at 185 Haig Road ,01-05 Singapore 114866 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 17, 'BN'), (95, 115, 'BA'), (116, 125, 'BC'), (126, 132, 'BP')]}), ('WHEREAS Intel has agreed to grant to the Licensee a Licence to use and occupy the space at 39 Keppel Road BLK 2 #02-04 Tanjong Pagar Distripark 2 Singapore 651574 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 13, 'BN'), (91, 145, 'BA'), (146, 155, 'BC'), (156, 162, 'BP')]}), ('WHEREAS HP has agreed to grant to the Licensee a Licence to use and occupy the space at 3026 Ubi Road 1 #04-160 Singapore 695006 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 10, 'BN'), (88, 111, 'BA'), (112, 121, 'BC'), (122, 128, 'BP')]}), ('WHEREAS Facebook has agreed to grant to the Licensee a Licence to use and occupy the space at 126B Dunlop Street Singapore 667212 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 16, 'BN'), (94, 112, 'BA'), (113, 122, 'BC'), (123, 129, 'BP')]}), ('WHEREAS LG Electronics has agreed to grant to the Licensee a Licence to use and occupy the space at 47 Jln Pemimpin #04-08 Singapore 130124 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 22, 'BN'), (100, 122, 'BA'), (123, 132, 'BC'), (133, 139, 'BP')]}), ('WHEREAS Lenovo has agreed to grant to the Licensee a Licence to use and occupy the space at 157 Yung Loh Road ,05 Singapore 820306 (herein referred to as "Premises") subject to the terms and conditions of this Agreement', {'entities': [(8, 14, 'BN'), (92, 113, 'BA'), (114, 123, 'BC'), (124, 130, 'BP')]}),
              ('This Licence Agreement is made on the hig day of spre mM ber - OS between 2C2P, Pyramid 87 Tuas Avenue 1 Singapore 555294 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 78, 'LN'), (80, 104, 'LA'), (105, 114, 'LC'), (115, 121, 'LP'), (126, 138, 'TN'), (164, 183, 'TA'), (184, 193, 'TC'), (194, 200, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Aetos Security Management, Blk 184 Toa Payoh Central #03-366 S() Singapore 636657 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 138, 'LA'), (139, 148, 'LC'), (149, 155, 'LP'), (160, 172, 'TN'), (198, 217, 'TA'), (218, 227, 'TC'), (228, 234, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between AIBI International, LOWER KENT RIDGE ROAD Singapore 722539 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 115, 'LA'), (116, 125, 'LC'), (126, 132, 'LP'), (137, 149, 'TN'), (175, 194, 'TA'), (195, 204, 'TC'), (205, 211, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Antlabs, Wing on Life Gardens 335 Bukit Timah Road #13-03 Singapore 588156 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 131, 'LA'), (132, 141, 'LC'), (142, 148, 'LP'), (153, 165, 'TN'), (191, 210, 'TA'), (211, 220, 'TC'), (221, 227, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Aspial Corporation, 32 Sungei Kadut Way HUA KOK INDUSTRIAL BUILDING Singapore 717318 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 141, 'LA'), (142, 151, 'LC'), (152, 158, 'LP'), (163, 175, 'TN'), (201, 220, 'TA'), (221, 230, 'TC'), (231, 237, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Ayam Brand, 222 222 Tagore Lane 02-11 Tg Building Singapore 305242 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 84, 'LN'), (86, 123, 'LA'), (124, 133, 'LC'), (134, 140, 'LP'), (145, 157, 'TN'), (183, 202, 'TA'), (203, 212, 'TC'), (213, 219, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Bee Cheng Hiang, 72 Bendemeer Rd #07-03 Luzerne S() Singapore 728742 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Boustead Singapore, 146 Pandan Loop Blk 10 Singapore 276403 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 116, 'LA'), (117, 126, 'LC'), (127, 133, 'LP'), (138, 150, 'TN'), (176, 195, 'TA'), (196, 205, 'TC'), (206, 212, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between BreadTalk, 2 Kallang Avenue 08-01/03 Kallang Bahru Complex Singapore 977633 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 132, 'LA'), (133, 142, 'LC'), (143, 149, 'LP'), (154, 166, 'TN'), (192, 211, 'TA'), (212, 221, 'TC'), (222, 228, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Broadcom Limited, Unit No.285, Kaki Bukit Singapore 627979 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 115, 'LA'), (116, 125, 'LC'), (126, 132, 'LP'), (137, 149, 'TN'), (175, 194, 'TA'), (195, 204, 'TC'), (205, 211, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between CapitaLand, Cairnhill Shop Houses 56 Cairnhill Road Singapore 899103 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 84, 'LN'), (86, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Carousell, Duchess Crest 58 Duchess Avenue #02-08 Singapore 837790 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 123, 'LA'), (124, 133, 'LC'), (134, 140, 'LP'), (145, 157, 'TN'), (183, 202, 'TA'), (203, 212, 'TC'), (213, 219, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Certis CISCO, CHANGI AIRPORT CARGO COMPLEX PO BOX 924 Singapore 532624 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 86, 'LN'), (88, 127, 'LA'), (128, 137, 'LC'), (138, 144, 'LP'), (149, 161, 'TN'), (187, 206, 'TA'), (207, 216, 'TC'), (217, 223, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Charles & Keith, 391A ORCHARD RD 12-06/07/08 NGEE ANN CITY TOWER A Singapore 952323 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 140, 'LA'), (141, 150, 'LC'), (151, 157, 'LP'), (162, 174, 'TN'), (200, 219, 'TA'), (220, 229, 'TC'), (230, 236, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between China Aviation Oil, 4 Sago Lane 02-111 Singapore 424787 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 112, 'LA'), (113, 122, 'LC'), (123, 129, 'LP'), (134, 146, 'TN'), (172, 191, 'TA'), (192, 201, 'TC'), (202, 208, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ComfortDelGro, Lucky Plaza 304 Orchard Road #03-43 Singapore 415540 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 87, 'LN'), (89, 124, 'LA'), (125, 134, 'LC'), (135, 141, 'LP'), (146, 158, 'TN'), (184, 203, 'TA'), (204, 213, 'TC'), (214, 220, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Creative Technology, 748 Yishun Street 72 01-212 Singapore 246659 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 93, 'LN'), (95, 122, 'LA'), (123, 132, 'LC'), (133, 139, 'LP'), (144, 156, 'TN'), (182, 201, 'TA'), (202, 211, 'TC'), (212, 218, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between dnata Singapore, Blk 602 AMK Ave 5 ,02-2655 Singapore 867869 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Far East Orchard, 448 Hougang Avenue 10 08-531 Singapore 420103 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 120, 'LA'), (121, 130, 'LC'), (131, 137, 'LP'), (142, 154, 'TN'), (180, 199, 'TA'), (200, 209, 'TC'), (210, 216, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Far East Organization, 100 High Street #06-01 The Treasury Singapore 682167 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 95, 'LN'), (97, 132, 'LA'), (133, 142, 'LC'), (143, 149, 'LP'), (154, 166, 'TN'), (192, 211, 'TA'), (212, 221, 'TC'), (222, 228, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between FilmTack, 10-12 SCOTTS ROAD, #02 Singapore 655576 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 82, 'LN'), (84, 106, 'LA'), (107, 116, 'LC'), (117, 123, 'LP'), (128, 140, 'TN'), (166, 185, 'TA'), (186, 195, 'TC'), (196, 202, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Flextronics, 175A Bencoolen Street #05-11 Office Tower Singapore 805448 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 85, 'LN'), (87, 128, 'LA'), (129, 138, 'LC'), (139, 145, 'LP'), (150, 162, 'TN'), (188, 207, 'TA'), (208, 217, 'TC'), (218, 224, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Fraser and Neave, 219 Henderson Road Singapore 649728 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 110, 'LA'), (111, 120, 'LC'), (121, 127, 'LP'), (132, 144, 'TN'), (170, 189, 'TA'), (190, 199, 'TC'), (200, 206, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Garena, 361 Ubi Road 3 #04-01 Singapore 136891 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 103, 'LA'), (104, 113, 'LC'), (114, 120, 'LP'), (125, 137, 'TN'), (163, 182, 'TA'), (183, 192, 'TC'), (193, 199, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Genting Singapore, 54 Genting Lane #01-04 Ruby Land Complex Blk 2 S() Singapore 569207 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 91, 'LN'), (93, 143, 'LA'), (144, 153, 'LC'), (154, 160, 'LP'), (165, 177, 'TN'), (203, 222, 'TA'), (223, 232, 'TC'), (233, 239, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between GlobalRoam, 331 NORTH BRIDGE ROAD 14-06 Singapore 210498 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 84, 'LN'), (86, 113, 'LA'), (114, 123, 'LC'), (124, 130, 'LP'), (135, 147, 'TN'), (173, 192, 'TA'), (193, 202, 'TC'), (203, 209, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Golden Agri-Resources, 1803 Geylang Bahru 01-01 Singapore 296457 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 95, 'LN'), (97, 121, 'LA'), (122, 131, 'LC'), (132, 138, 'LP'), (143, 155, 'TN'), (181, 200, 'TA'), (201, 210, 'TC'), (211, 217, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Grab, 154 West Coast Road 01-115 Ginza Plaza Singapore 919784 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 78, 'LN'), (80, 118, 'LA'), (119, 128, 'LC'), (129, 135, 'LP'), (140, 152, 'TN'), (178, 197, 'TA'), (198, 207, 'TC'), (208, 214, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Great Eastern Life, Hup Lian Huat Engineering 38 Defu Lane 9 Singapore 223316 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 134, 'LA'), (135, 144, 'LC'), (145, 151, 'LP'), (156, 168, 'TN'), (194, 213, 'TA'), (214, 223, 'TC'), (224, 230, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Hyflux, Far East Shopping Centre 545 Orchard Road #04-14 Singapore 892492 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 130, 'LA'), (131, 140, 'LC'), (141, 147, 'LP'), (152, 164, 'TN'), (190, 209, 'TA'), (210, 219, 'TC'), (220, 226, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Jetstar Asia Airways, 21 Senoko South Road Woodlands East Singapore 965946 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 94, 'LN'), (96, 131, 'LA'), (132, 141, 'LC'), (142, 148, 'LP'), (153, 165, 'TN'), (191, 210, 'TA'), (211, 220, 'TC'), (221, 227, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between JTC Corporation, International Building 360 Orchard Road #12-04 Singapore 305386 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Jurong Port, Ion Orchard, 2 Orchard Turn, Singapore 763791 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 85, 'LN'), (87, 115, 'LA'), (116, 125, 'LC'), (126, 132, 'LP'), (137, 149, 'TN'), (175, 194, 'TA'), (195, 204, 'TC'), (205, 211, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Keppel Corporation, 25 Lorong Kilat #02-03 Singapore 343160 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 116, 'LA'), (117, 126, 'LC'), (127, 133, 'LP'), (138, 150, 'TN'), (176, 195, 'TA'), (196, 205, 'TC'), (206, 212, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Koufu, 20 Ang Mo Kio Industrial Park 2A 02-02 Singapore 693818 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 79, 'LN'), (81, 119, 'LA'), (120, 129, 'LC'), (130, 136, 'LP'), (141, 153, 'TN'), (179, 198, 'TA'), (199, 208, 'TC'), (209, 215, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between M1 Limited, Paramount Shopping Centre 30 East Coast Road #02-08 Singapore 147305 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 84, 'LN'), (86, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Mediacorp, 1 Finlayson Green #13-00 Singapore 425984 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 109, 'LA'), (110, 119, 'LC'), (120, 126, 'LP'), (131, 143, 'TN'), (169, 188, 'TA'), (189, 198, 'TC'), (199, 205, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between MyRepublic, 20 Tuas Avenue 8 Singapore 435720 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 84, 'LN'), (86, 102, 'LA'), (103, 112, 'LC'), (113, 119, 'LP'), (124, 136, 'TN'), (162, 181, 'TA'), (182, 191, 'TC'), (192, 198, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Neptune Orient Lines, 57 UBI AVENUE 1, #05-14 Singapore 778909 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 94, 'LN'), (96, 119, 'LA'), (120, 129, 'LC'), (130, 136, 'LP'), (141, 153, 'TN'), (179, 198, 'TA'), (199, 208, 'TC'), (209, 215, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NETS, 501 Bukit Timah Road #01-05B Singapore 620727 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 78, 'LN'), (80, 108, 'LA'), (109, 118, 'LC'), (119, 125, 'LP'), (130, 142, 'TN'), (168, 187, 'TA'), (188, 197, 'TC'), (198, 204, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NTUC FairPrice, HDB Jurong 351 Jurong East Street 31 #01-81 Singapore 594529 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 88, 'LN'), (90, 133, 'LA'), (134, 143, 'LC'), (144, 150, 'LP'), (155, 167, 'TN'), (193, 212, 'TA'), (213, 222, 'TC'), (223, 229, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between OCBC Bank, 32 Defu Lane 10 04-34 Singapore 915498 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 106, 'LA'), (107, 116, 'LC'), (117, 123, 'LP'), (128, 140, 'TN'), (166, 185, 'TA'), (186, 195, 'TC'), (196, 202, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Osim International, Haw Par Technocentre #01-04, 401 Commonwealth Drive Singapore 293654 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 145, 'LA'), (146, 155, 'LC'), (156, 162, 'LP'), (167, 179, 'TN'), (205, 224, 'TA'), (225, 234, 'TC'), (235, 241, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Pacific Century Regional Developments Limited, 8 Jalan Kilang Timor #02-01 KEWALRAM HOUSE Singapore 862444 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 119, 'LN'), (121, 163, 'LA'), (164, 173, 'LC'), (174, 180, 'LP'), (185, 197, 'TN'), (223, 242, 'TA'), (243, 252, 'TC'), (253, 259, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Popular Holdings, 166 Bukit Merah Central 01-3531 Singapore 102127 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 123, 'LA'), (124, 133, 'LC'), (134, 140, 'LP'), (145, 157, 'TN'), (183, 202, 'TA'), (203, 212, 'TC'), (213, 219, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between POSB Bank, Changi Airport Terminal 1, 80 Airport Boulevard, Singapore 954993 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 133, 'LA'), (134, 143, 'LC'), (144, 150, 'LP'), (155, 167, 'TN'), (193, 212, 'TA'), (213, 222, 'TC'), (223, 229, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between PSA International, 27 Jln Melor Singapore 877704 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 91, 'LN'), (93, 105, 'LA'), (106, 115, 'LC'), (116, 122, 'LP'), (127, 139, 'TN'), (165, 184, 'TA'), (185, 194, 'TC'), (195, 201, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Quest Global, 1002 Jln Bt Merah #07-11 Redhill Ind Est S() Singapore 127915 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 86, 'LN'), (88, 132, 'LA'), (133, 142, 'LC'), (143, 149, 'LP'), (154, 166, 'TN'), (192, 211, 'TA'), (212, 221, 'TC'), (222, 228, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Renewable Energy Corporation, 163 Tanglin Road 03-19 Tanglin Mall Singapore 255911 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 139, 'LA'), (140, 149, 'LC'), (150, 156, 'LP'), (161, 173, 'TN'), (199, 218, 'TA'), (219, 228, 'TC'), (229, 235, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Robinsons & Co., 354 Clementi Ave 2 #01-253 Singapore 657574 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SATS Ltd, 9010 Tampines St 93 #04-109 Singapore 118699 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 82, 'LN'), (84, 111, 'LA'), (112, 121, 'LC'), (122, 128, 'LP'), (133, 145, 'TN'), (171, 190, 'TA'), (191, 200, 'TC'), (201, 207, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SBS Transit, Bedok Clinic & Surgery 17 Bedok South Road #01-581 Singapore 333904 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 85, 'LN'), (87, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Scoot, 1 Coleman Street 02-38 Adelphi The Singapore 375835 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 79, 'LN'), (81, 115, 'LA'), (116, 125, 'LC'), (126, 132, 'LP'), (137, 149, 'TN'), (175, 194, 'TA'), (195, 204, 'TC'), (205, 211, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SembCorp Marine, 230 Orchard Road #10-230 Faber House Singapore 659453 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 127, 'LA'), (128, 137, 'LC'), (138, 144, 'LP'), (149, 161, 'TN'), (187, 206, 'TA'), (207, 216, 'TC'), (217, 223, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SGAG, 583 ORCHARD ROAD, #14-02 Singapore 841193 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 78, 'LN'), (80, 104, 'LA'), (105, 114, 'LC'), (115, 121, 'LP'), (126, 138, 'TN'), (164, 183, 'TA'), (184, 193, 'TC'), (194, 200, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Sheng Siong, The Paterson Edge 26 Paterson Road #01-00 Singapore 947834 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 85, 'LN'), (87, 128, 'LA'), (129, 138, 'LC'), (139, 145, 'LP'), (150, 162, 'TN'), (188, 207, 'TA'), (208, 217, 'TC'), (218, 224, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Shopee, 10 Admiralty St #05-25 North Link Bldg S() Singapore 806333 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 124, 'LA'), (125, 134, 'LC'), (135, 141, 'LP'), (146, 158, 'TN'), (184, 203, 'TA'), (204, 213, 'TC'), (214, 220, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SIA Engineering Company, 10 Anson Road 11-10 International Plaza Singapore 933044 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 97, 'LN'), (99, 138, 'LA'), (139, 148, 'LC'), (149, 155, 'LP'), (160, 172, 'TN'), (198, 217, 'TA'), (218, 227, 'TC'), (228, 234, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SilkAir, BLK 231 BISHAN STREET 23#B1-05 Singapore 262052 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 113, 'LA'), (114, 123, 'LC'), (124, 130, 'LP'), (135, 147, 'TN'), (173, 192, 'TA'), (193, 202, 'TC'), (203, 209, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Airlines, BLK 682 HOUGANG AVENUE 4#01-322 Singapore 922699 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Airlines Cargo, 143 Cecil Street 18-00 Gb Building Singapore 454787 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 98, 'LN'), (100, 134, 'LA'), (135, 144, 'LC'), (145, 151, 'LP'), (156, 168, 'TN'), (194, 213, 'TA'), (214, 223, 'TC'), (224, 230, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Exchange, 1D Yong Siak Street Singapore 602875 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 92, 'LN'), (94, 113, 'LA'), (114, 123, 'LC'), (124, 130, 'LP'), (135, 147, 'TN'), (173, 192, 'TA'), (193, 202, 'TC'), (203, 209, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Petroleum Company Limited, 21, Kings Polemeller Plaza, Smeily Street Singapore 826024 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 109, 'LN'), (111, 152, 'LA'), (153, 162, 'LC'), (163, 169, 'LP'), (174, 186, 'TN'), (212, 231, 'TA'), (232, 241, 'TC'), (242, 248, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Post, 09-13 ALEXANDRA DISTRIPARK BLK 4 Singapore 453838 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 88, 'LN'), (90, 122, 'LA'), (123, 132, 'LC'), (133, 139, 'LP'), (144, 156, 'TN'), (182, 201, 'TA'), (202, 211, 'TC'), (212, 218, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singapore Press Holdings, 10 Shenton Way 11-03 M A S Building Singapore 745153 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 98, 'LN'), (100, 135, 'LA'), (136, 145, 'LC'), (146, 152, 'LP'), (157, 169, 'TN'), (195, 214, 'TA'), (215, 224, 'TC'), (225, 231, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Singtel, 210 Middle Road #11-06 IOI Plaza Singapore 305480 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 115, 'LA'), (116, 125, 'LC'), (126, 132, 'LP'), (137, 149, 'TN'), (175, 194, 'TA'), (195, 204, 'TC'), (205, 211, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SMRT Corporation, 3 Pasir Panjang Rd Alexandra Distripark Singapore 431905 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 131, 'LA'), (132, 141, 'LC'), (142, 148, 'LP'), (153, 165, 'TN'), (191, 210, 'TA'), (211, 220, 'TC'), (221, 227, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SP Group, HDB Pasir Ris 419 Pasir Ris Drive 6 #08-275 Singapore 687419 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 82, 'LN'), (84, 127, 'LA'), (128, 137, 'LC'), (138, 144, 'LP'), (149, 161, 'TN'), (187, 206, 'TA'), (207, 216, 'TC'), (217, 223, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ST Engineering, 85 Science Park Drive, ,01-01&01-02 The Cavendish Singapore 412539 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 88, 'LN'), (90, 139, 'LA'), (140, 149, 'LC'), (150, 156, 'LP'), (161, 173, 'TN'), (199, 218, 'TA'), (219, 228, 'TC'), (229, 235, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Standard Chartered Singapore, 115 AIRPORT CARGO ROAD 06-18 CARGO AGENTS BUILDING C Singapore 470174 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 156, 'LA'), (157, 166, 'LC'), (167, 173, 'LP'), (178, 190, 'TN'), (216, 235, 'TA'), (236, 245, 'TC'), (246, 252, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between StarHub, 72 Bendemeer Road #03-31 Luzerne (Formerly Known As Hiap Huat House) Singapore 919746 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 151, 'LA'), (152, 161, 'LC'), (162, 168, 'LP'), (173, 185, 'TN'), (211, 230, 'TA'), (231, 240, 'TC'), (241, 247, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Systems on Silicon Manufacturing, 1 Scotts Road #03-04 Shaw Centre Singapore 621306 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 106, 'LN'), (108, 140, 'LA'), (141, 150, 'LC'), (151, 157, 'LP'), (162, 174, 'TN'), (200, 219, 'TA'), (220, 229, 'TC'), (230, 236, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Tangs, 25 INTERNATIONAL BUSINESS PARK 03-20 GERMAN CENTRE Singapore 538582 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 79, 'LN'), (81, 131, 'LA'), (132, 141, 'LC'), (142, 148, 'LP'), (153, 165, 'TN'), (191, 210, 'TA'), (211, 220, 'TC'), (221, 227, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Tee Yih Jia, 49 Jalan Pemimpin #05-04 Aps Industrial Building Singapore 160535 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 85, 'LN'), (87, 135, 'LA'), (136, 145, 'LC'), (146, 152, 'LP'), (157, 169, 'TN'), (195, 214, 'TA'), (215, 224, 'TC'), (225, 231, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Temasek Holdings, 201 Keppel Road Blk 6 Singapore 693540 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 90, 'LN'), (92, 113, 'LA'), (114, 123, 'LC'), (124, 130, 'LP'), (135, 147, 'TN'), (173, 192, 'TA'), (193, 202, 'TC'), (203, 209, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Thakral Corporation, 7 Tuas Ave 2 Singapore 494280 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 93, 'LN'), (95, 107, 'LA'), (108, 117, 'LC'), (118, 124, 'LP'), (129, 141, 'TN'), (167, 186, 'TA'), (187, 196, 'TC'), (197, 203, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Tiger Airways Holdings, 9 Lorong 21A Geylang #03-00 Singapore 550090 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 96, 'LN'), (98, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Twelve Cupcakes, 2 Toh Tuck Link #03-00 S() Singapore 952049 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 89, 'LN'), (91, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Vertex Venture Holdings, 18 Sungei Tengah Road Singapore 799837 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 97, 'LN'), (99, 120, 'LA'), (121, 130, 'LC'), (131, 137, 'LP'), (142, 154, 'TN'), (180, 199, 'TA'), (200, 209, 'TC'), (210, 216, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Wilmar, 51 Thomson Road #181B Goldhill Centre Singapore 129211 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 119, 'LA'), (120, 129, 'LC'), (130, 136, 'LP'), (141, 153, 'TN'), (179, 198, 'TA'), (199, 208, 'TC'), (209, 215, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Ya Kun Kaya Toast, 504 JURONG WEST STREET 51 02-213 Singapore 252215 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 91, 'LN'), (93, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Yeo Hiap Seng, 201E TAMPINES STREET 23 01-72 Singapore 664888 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 87, 'LN'), (89, 118, 'LA'), (119, 128, 'LC'), (129, 135, 'LP'), (140, 152, 'TN'), (178, 197, 'TA'), (198, 207, 'TC'), (208, 214, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ACCOUNTING AND CORPORATE REGULATORY AUTHORITY (ACRA), Guan Hua Warehouse Building 85 Genting Lane #02-02 Singapore 420087 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 126, 'LN'), (128, 178, 'LA'), (179, 188, 'LC'), (189, 195, 'LP'), (200, 212, 'TN'), (238, 257, 'TA'), (258, 267, 'TC'), (268, 274, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between AGENCY FOR SCIENCE, TECHNOLOGY AND RESEARCH (A*STAR), 1 FISHERY PORT ROAD Singapore 768852 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 126, 'LN'), (128, 147, 'LA'), (148, 157, 'LC'), (158, 164, 'LP'), (169, 181, 'TN'), (207, 226, 'TA'), (227, 236, 'TC'), (237, 243, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between BOARD OF ARCHITECTS (BOA), 331 North Bridge Road ,04-06 Odeon Towers Singapore 699894 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 142, 'LA'), (143, 152, 'LC'), (153, 159, 'LP'), (164, 176, 'TN'), (202, 221, 'TA'), (222, 231, 'TC'), (232, 238, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between BUILDING AND CONSTRUCTION AUTHORITY (BCA), 116 CHANGI ROAD, #02-05 Singapore 655392 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 115, 'LN'), (117, 140, 'LA'), (141, 150, 'LC'), (151, 157, 'LP'), (162, 174, 'TN'), (200, 219, 'TA'), (220, 229, 'TC'), (230, 236, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between CASINO REGULATORY AUTHORITY OF SINGAPORE (CRA), 6 BATTERY RD 11-07 Singapore 842796 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 120, 'LN'), (122, 140, 'LA'), (141, 150, 'LC'), (151, 157, 'LP'), (162, 174, 'TN'), (200, 219, 'TA'), (220, 229, 'TC'), (230, 236, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between CENTRAL PROVIDENT FUND BOARD (CPFB), 160 Paya Lebar Road 07-08 Orion Industrial Building Singapore 216383 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 109, 'LN'), (111, 162, 'LA'), (163, 172, 'LC'), (173, 179, 'LP'), (184, 196, 'TN'), (222, 241, 'TA'), (242, 251, 'TC'), (252, 258, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between CIVIL AVIATION AUTHORITY OF SINGAPORE (CAAS), 304 Orchard Rd #03-62 Singapore 161192 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 118, 'LN'), (120, 141, 'LA'), (142, 151, 'LC'), (152, 158, 'LP'), (163, 175, 'TN'), (201, 220, 'TA'), (221, 230, 'TC'), (231, 237, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between CIVIL SERVICE COLLEGE (CSC), 11 Berwick Dr S() Singapore 289008 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 101, 'LN'), (103, 120, 'LA'), (121, 130, 'LC'), (131, 137, 'LP'), (142, 154, 'TN'), (180, 199, 'TA'), (200, 209, 'TC'), (210, 216, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between COMPETITION AND CONSUMER COMMISSION OF SINGAPORE (CCCS), No.2 Bukit Merah Central, #10-00 Spring Building, 5 Singapore 980783 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 129, 'LN'), (131, 182, 'LA'), (183, 192, 'LC'), (193, 199, 'LP'), (204, 216, 'TN'), (242, 261, 'TA'), (262, 271, 'TC'), (272, 278, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between COUNCIL FOR ESTATE AGENCIES (CEA), 3 ROCHOR ROAD 02-628 ROCHOR CENTRE Singapore 281201 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 107, 'LN'), (109, 143, 'LA'), (144, 153, 'LC'), (154, 160, 'LP'), (165, 177, 'TN'), (203, 222, 'TA'), (223, 232, 'TC'), (233, 239, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between DEFENCE SCIENCE AND TECHNOLOGY AGENCY (DSTA), 2 Haig Rd #04-517 Singapore 718374 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 118, 'LN'), (120, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ECONOMIC DEVELOPMENT BOARD (EDB), 400 ORCHARD ROAD, #03-11 Singapore 323532 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 106, 'LN'), (108, 132, 'LA'), (133, 142, 'LC'), (143, 149, 'LP'), (154, 166, 'TN'), (192, 211, 'TA'), (212, 221, 'TC'), (222, 228, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ENERGY MARKET AUTHORITY (EMA), 3 Temasek Boulevard B1-024 Suntec City Mall Singapore 212321 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 103, 'LN'), (105, 148, 'LA'), (149, 158, 'LC'), (159, 165, 'LP'), (170, 182, 'TN'), (208, 227, 'TA'), (228, 237, 'TC'), (238, 244, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ENTERPRISE SINGAPORE (ESG), 446 Bright Hill Drive 02-111 Singapore 861237 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 100, 'LN'), (102, 130, 'LA'), (131, 140, 'LC'), (141, 147, 'LP'), (152, 164, 'TN'), (190, 209, 'TA'), (210, 219, 'TC'), (220, 226, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between GOVERNMENT TECHNOLOGY AGENCY (GOVTECH), 111 North Bridge Road #27-01 Peninsula Plaza Singapore 218131 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 112, 'LN'), (114, 158, 'LA'), (159, 168, 'LC'), (169, 175, 'LP'), (180, 192, 'TN'), (218, 237, 'TA'), (238, 247, 'TC'), (248, 254, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HEALTH PROMOTION BOARD (HPB), 2 Bayfront Avenue, #B1-60 Singapore 310079 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 129, 'LA'), (130, 139, 'LC'), (140, 146, 'LP'), (151, 163, 'TN'), (189, 208, 'TA'), (209, 218, 'TC'), (219, 225, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HEALTH SCIENCES AUTHORITY (HSA), 767 Geylang Road Singapore 454106 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 105, 'LN'), (107, 123, 'LA'), (124, 133, 'LC'), (134, 140, 'LP'), (145, 157, 'TN'), (183, 202, 'TA'), (203, 212, 'TC'), (213, 219, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HOME TEAM SCIENCE AND TECHNOLOGY AGENCY (HTX), 8 Loyang Drive Singapore 623578 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 119, 'LN'), (121, 135, 'LA'), (136, 145, 'LC'), (146, 152, 'LP'), (157, 169, 'TN'), (195, 214, 'TA'), (215, 224, 'TC'), (225, 231, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HOTELS LICENSING BOARD (HLB), No. 1 Marine Parade Central, 12-07 Parway Centre Singapore 927292 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 152, 'LA'), (153, 162, 'LC'), (163, 169, 'LP'), (174, 186, 'TN'), (212, 231, 'TA'), (232, 241, 'TC'), (242, 248, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HOUSING & DEVELOPMENT BOARD (HDB), 30 Bukit Batok Street 31 #04-01 Hong Kah North Community Centre Singapore 742417 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 107, 'LN'), (109, 172, 'LA'), (173, 182, 'LC'), (183, 189, 'LP'), (194, 206, 'TN'), (232, 251, 'TA'), (252, 261, 'TC'), (262, 268, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between INFO-COMMUNICATIONS MEDIA DEVELOPMENT AUTHORITY (IMDA), 6A Napier Road #02-38 Gleneagles Hospital Singapore 182472 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 128, 'LN'), (130, 171, 'LA'), (172, 181, 'LC'), (182, 188, 'LP'), (193, 205, 'TN'), (231, 250, 'TA'), (251, 260, 'TC'), (261, 267, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between INLAND REVENUE AUTHORITY OF SINGAPORE (IRAS), Blk 20 Ang Mo Kio Industrial Park 2A ,05-11 AMK Techlink Singapore 226885 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 118, 'LN'), (120, 176, 'LA'), (177, 186, 'LC'), (187, 193, 'LP'), (198, 210, 'TN'), (236, 255, 'TA'), (256, 265, 'TC'), (266, 272, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between INSTITUTE OF TECHNICAL EDUCATION (ITE), 27 Tuas Avenue 13 01-22 Singapore 893447 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 112, 'LN'), (114, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between INTELLECTUAL PROPERTY OFFICE OF SINGAPORE (IPOS), Lorong 1 Toa Payoh, Oleander Towers Singapore 629739 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 122, 'LN'), (124, 159, 'LA'), (160, 169, 'LC'), (170, 176, 'LP'), (181, 193, 'TN'), (219, 238, 'TA'), (239, 248, 'TC'), (249, 255, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between ISEAS-YUSOF ISHAK INSTITUTE (ISEAS), 5 SHENTON WAY, #08-02 Singapore 915879 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 109, 'LN'), (111, 132, 'LA'), (133, 142, 'LC'), (143, 149, 'LP'), (154, 166, 'TN'), (192, 211, 'TA'), (212, 221, 'TC'), (222, 228, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between JTC CORPORATION (JTC), Blk 9 #01-12 Dempsey Road, Dempsey Hill Singapore 351481 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 95, 'LN'), (97, 136, 'LA'), (137, 146, 'LC'), (147, 153, 'LP'), (158, 170, 'TN'), (196, 215, 'TA'), (216, 225, 'TC'), (226, 232, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between LAND SURVEYORS BOARD (LSB), 9B Hoe Chiang Road Singapore 548212 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 100, 'LN'), (102, 120, 'LA'), (121, 130, 'LC'), (131, 137, 'LP'), (142, 154, 'TN'), (180, 199, 'TA'), (200, 209, 'TC'), (210, 216, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between LAND TRANSPORT AUTHORITY (LTA), 42 Woodlands Terrace Singapore 614655 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 104, 'LN'), (106, 126, 'LA'), (127, 136, 'LC'), (137, 143, 'LP'), (148, 160, 'TN'), (186, 205, 'TA'), (206, 215, 'TC'), (216, 222, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between MAJLIS UGAMA ISLAM SINGAPURA (MUIS), 304 Orchard Road 01-53 Lucky Plaza Singapore 478662 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 109, 'LN'), (111, 145, 'LA'), (146, 155, 'LC'), (156, 162, 'LP'), (167, 179, 'TN'), (205, 224, 'TA'), (225, 234, 'TC'), (235, 241, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between MARITIME AND PORT AUTHORITY OF SINGAPORE (MPA), BLK 105 ANG MO KIO AVENUE 4, #06-194 Singapore 728841 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 120, 'LN'), (122, 158, 'LA'), (159, 168, 'LC'), (169, 175, 'LP'), (180, 192, 'TN'), (218, 237, 'TA'), (238, 247, 'TC'), (248, 254, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between MONETARY AUTHORITY OF SINGAPORE (MAS), 30 HONGKONG STREET Singapore 743828 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 111, 'LN'), (113, 131, 'LA'), (132, 141, 'LC'), (142, 148, 'LP'), (153, 165, 'TN'), (191, 210, 'TA'), (211, 220, 'TC'), (221, 227, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NANYANG POLYTECHNIC (NYP), 4 Kwong Min Road Singapore 559336 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL ARTS COUNCIL (NAC), LCK25 LIM CHU KANG LANE 3 Singapore 647406 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 101, 'LN'), (103, 128, 'LA'), (129, 138, 'LC'), (139, 145, 'LP'), (150, 162, 'TN'), (188, 207, 'TA'), (208, 217, 'TC'), (218, 224, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL COUNCIL OF SOCIAL SERVICE (NCSS), 360 Orchard Road #09-01 INTERNATIONAL BUILDING Singapore 838030 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 115, 'LN'), (117, 163, 'LA'), (164, 173, 'LC'), (174, 180, 'LP'), (185, 197, 'TN'), (223, 242, 'TA'), (243, 252, 'TC'), (253, 259, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL ENVIRONMENT AGENCY (NEA), 420 North Bridge Road #03-07 North Bridge Centre Singapore 769977 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 107, 'LN'), (109, 157, 'LA'), (158, 167, 'LC'), (168, 174, 'LP'), (179, 191, 'TN'), (217, 236, 'TA'), (237, 246, 'TC'), (247, 253, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL HERITAGE BOARD (NHB), 364 JOO CHIAT ROAD Singapore 239679 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 103, 'LN'), (105, 123, 'LA'), (124, 133, 'LC'), (134, 140, 'LP'), (145, 157, 'TN'), (183, 202, 'TA'), (203, 212, 'TC'), (213, 219, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL LIBRARY BOARD (NLB), 89 Circuit Rd #01-941A Singapore 991833 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 126, 'LA'), (127, 136, 'LC'), (137, 143, 'LP'), (148, 160, 'TN'), (186, 205, 'TA'), (206, 215, 'TC'), (216, 222, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NATIONAL PARKS BOARD (NPARKS), lot lck 34, 31 lim chu kang lane 6c, Singapore 132761 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 103, 'LN'), (105, 141, 'LA'), (142, 151, 'LC'), (152, 158, 'LP'), (163, 175, 'TN'), (201, 220, 'TA'), (221, 230, 'TC'), (231, 237, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between NGEE ANN POLYTECHNIC (NP), 9, Jalan Pinang, Singapore 158355 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ("This Licence Agreement is made on the hig day of spre mM ber - OS between PEOPLE'S ASSOCIATION (PA), 605B Macpherson Road #04-13 Citimac Industrial Complex B Singapore 661924 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982", {'entities': [(74, 99, 'LN'), (101, 157, 'LA'), (158, 167, 'LC'), (168, 174, 'LP'), (179, 191, 'TN'), (217, 236, 'TA'), (237, 246, 'TC'), (247, 253, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between PROFESSIONAL ENGINEERS BOARD, SINGAPORE (PEB), Blk 244 Jurong East St 24 Singapore 125016 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 119, 'LN'), (121, 146, 'LA'), (147, 156, 'LC'), (157, 163, 'LP'), (168, 180, 'TN'), (206, 225, 'TA'), (226, 235, 'TC'), (236, 242, 'TP')]}), ("This Licence Agreement is made on the hig day of spre mM ber - OS between PUB, Singapore's National Water Agency (PUB), Tampines St 41, Tampines Singapore 850343 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982", {'entities': [(74, 118, 'LN'), (120, 144, 'LA'), (145, 154, 'LC'), (155, 161, 'LP'), (166, 178, 'TN'), (204, 223, 'TA'), (224, 233, 'TC'), (234, 240, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between PUBLIC TRANSPORT COUNCIL (PTC), 60 COLLYER QUAY 32 Singapore 619186 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 104, 'LN'), (106, 124, 'LA'), (125, 134, 'LC'), (135, 141, 'LP'), (146, 158, 'TN'), (184, 203, 'TA'), (204, 213, 'TC'), (214, 220, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between REPUBLIC POLYTECHNIC (RP), BLK 119 ALJUNIED AVENUE 2, #01-K4 Singapore 814692 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 134, 'LA'), (135, 144, 'LC'), (145, 151, 'LP'), (156, 168, 'TN'), (194, 213, 'TA'), (214, 223, 'TC'), (224, 230, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SCIENCE CENTRE BOARD (SCB), 11 Haji Lane S() Singapore 269539 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 100, 'LN'), (102, 118, 'LA'), (119, 128, 'LC'), (129, 135, 'LP'), (140, 152, 'TN'), (178, 197, 'TA'), (198, 207, 'TC'), (208, 214, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SENTOSA DEVELOPMENT CORPORATION (SDC), Blk 128 Kim Tian Road# 16-119 Singapore 796865 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 111, 'LN'), (113, 142, 'LA'), (143, 152, 'LC'), (153, 159, 'LP'), (164, 176, 'TN'), (202, 221, 'TA'), (222, 231, 'TC'), (232, 238, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE ACCOUNTANCY COMMISSION (SAC), 9 Kaki Bukit Rd 2 #01-09 Singapore 391920 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 112, 'LN'), (114, 138, 'LA'), (139, 148, 'LC'), (149, 155, 'LP'), (160, 172, 'TN'), (198, 217, 'TA'), (218, 227, 'TC'), (228, 234, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE DENTAL COUNCIL (SDC), 275 Thomson Road 01-10 Novena Ville Singapore 512171 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 104, 'LN'), (106, 141, 'LA'), (142, 151, 'LC'), (152, 158, 'LP'), (163, 175, 'TN'), (201, 220, 'TA'), (221, 230, 'TC'), (231, 237, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE EXAMINATIONS AND ASSESSMENT BOARD (SEAB), Suntec Tower Two 9 Temasek Boulevard #04-01 Singapore 546462 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 124, 'LN'), (126, 169, 'LA'), (170, 179, 'LC'), (180, 186, 'LP'), (191, 203, 'TN'), (229, 248, 'TA'), (249, 258, 'TC'), (259, 265, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE FOOD AGENCY (SFA), 144 UPPER BUKIT TIMAH ROAD, #02-17 Singapore 575841 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 101, 'LN'), (103, 137, 'LA'), (138, 147, 'LC'), (148, 154, 'LP'), (159, 171, 'TN'), (197, 216, 'TA'), (217, 226, 'TC'), (227, 233, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE LABOUR FOUNDATION (SLF), #04-33V, IMM Building, No.2 Jurong East Street 21 Singapore 526435 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 107, 'LN'), (109, 158, 'LA'), (159, 168, 'LC'), (169, 175, 'LP'), (180, 192, 'TN'), (218, 237, 'TA'), (238, 247, 'TC'), (248, 254, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE LAND AUTHORITY (SLA), 49 TANNERY LANE 05-03 Singapore 448638 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 104, 'LN'), (106, 127, 'LA'), (128, 137, 'LC'), (138, 144, 'LP'), (149, 161, 'TN'), (187, 206, 'TA'), (207, 216, 'TC'), (217, 223, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE MEDICAL COUNCIL (SMC), 1 1 North Bridge Road 01-01 High St Centre Singapore 334252 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 105, 'LN'), (107, 149, 'LA'), (150, 159, 'LC'), (160, 166, 'LP'), (171, 183, 'TN'), (209, 228, 'TA'), (229, 238, 'TC'), (239, 245, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE NURSING BOARD (SNB), 290 Orchard Rd #09-07 Singapore 134328 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 103, 'LN'), (105, 126, 'LA'), (127, 136, 'LC'), (137, 143, 'LP'), (148, 160, 'TN'), (186, 205, 'TA'), (206, 215, 'TC'), (216, 222, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE PHARMACY COUNCIL (SPC), Fortune Centre, 190 Middle Road, #10-08 Singapore 609242 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 106, 'LN'), (108, 147, 'LA'), (148, 157, 'LC'), (158, 164, 'LP'), (169, 181, 'TN'), (207, 226, 'TA'), (227, 236, 'TC'), (237, 243, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE POLYTECHNIC (SP), 51 Lorong 3 Geylang 05-01 Singapore 728670 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 100, 'LN'), (102, 127, 'LA'), (128, 137, 'LC'), (138, 144, 'LP'), (149, 161, 'TN'), (187, 206, 'TA'), (207, 216, 'TC'), (217, 223, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SINGAPORE TOURISM BOARD (STB), 123 McNair Rd #10-07 Singapore 281874 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 103, 'LN'), (105, 125, 'LA'), (126, 135, 'LC'), (136, 142, 'LP'), (147, 159, 'TN'), (185, 204, 'TA'), (205, 214, 'TC'), (215, 221, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SKILLSFUTURE SINGAPORE (SSG), 20 Martin Road #08-01 SENG KEE BUILDING Singapore 535086 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 102, 'LN'), (104, 143, 'LA'), (144, 153, 'LC'), (154, 160, 'LP'), (165, 177, 'TN'), (203, 222, 'TA'), (223, 232, 'TC'), (233, 239, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between SPORT SINGAPORE (SportSG), 252 North Bridge Road Singapore 149607 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 122, 'LA'), (123, 132, 'LC'), (133, 139, 'LP'), (144, 156, 'TN'), (182, 201, 'TA'), (202, 211, 'TC'), (212, 218, 'TP')]}), ("This Licence Agreement is made on the hig day of spre mM ber - OS between TEMASEK POLYTECHNIC (TP), 101 UPPER CROSS ST B1-03 PEOPLE'S PARK CENTRE Singapore 666832 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982", {'entities': [(74, 98, 'LN'), (100, 145, 'LA'), (146, 155, 'LC'), (156, 162, 'LP'), (167, 179, 'TN'), (205, 224, 'TA'), (225, 234, 'TC'), (235, 241, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between TOTE BOARD (TOTE BOARD), 140 Robinson Road, #03-05, Chow House Singapore 973023 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 97, 'LN'), (99, 136, 'LA'), (137, 146, 'LC'), (147, 153, 'LP'), (158, 170, 'TN'), (196, 215, 'TA'), (216, 225, 'TC'), (226, 232, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between TRADITIONAL CHINESE MEDICINE PRACTITIONERS BOARD (TCMPB), 32 Quality Road Singapore 224567 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 130, 'LN'), (132, 147, 'LA'), (148, 157, 'LC'), (158, 164, 'LP'), (169, 181, 'TN'), (207, 226, 'TA'), (227, 236, 'TC'), (237, 243, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between URBAN REDEVELOPMENT AUTHORITY (URA), 5 Kaki Bukit Road 2 #01-10 & #02-10 CITY WAREHOUSE Singapore 852714 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 109, 'LN'), (111, 161, 'LA'), (162, 171, 'LC'), (172, 178, 'LP'), (183, 195, 'TN'), (221, 240, 'TA'), (241, 250, 'TC'), (251, 257, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between WORKFORCE SINGAPORE (WSG), 24 Raffles Place 19-04 Clifford Centre Singapore 320380 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 99, 'LN'), (101, 139, 'LA'), (140, 149, 'LC'), (150, 156, 'LP'), (161, 173, 'TN'), (199, 218, 'TA'), (219, 228, 'TC'), (229, 235, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between YELLOW RIBBON SINGAPORE (YRSG), 17 Soon Lee Road Singapore 387438 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 104, 'LN'), (106, 122, 'LA'), (123, 132, 'LC'), (133, 139, 'LP'), (144, 156, 'TN'), (182, 201, 'TA'), (202, 211, 'TC'), (212, 218, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Samsung Electronics, 3007 Ubi Road 1 05-400 Singapore 770349 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 93, 'LN'), (95, 117, 'LA'), (118, 127, 'LC'), (128, 134, 'LP'), (139, 151, 'TN'), (177, 196, 'TA'), (197, 206, 'TC'), (207, 213, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Foxconn, 155 Kew Cres S() Singapore 941272 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 99, 'LA'), (100, 109, 'LC'), (110, 116, 'LP'), (121, 133, 'TN'), (159, 178, 'TA'), (179, 188, 'TC'), (189, 195, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Alphabet, 7 Wholesales Centre #01-235 Singapore 560593 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 82, 'LN'), (84, 111, 'LA'), (112, 121, 'LC'), (122, 128, 'LP'), (133, 145, 'TN'), (171, 190, 'TA'), (191, 200, 'TC'), (201, 207, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Microsoft, 391 ORCHARD ROAD, #03-51 Singapore 997937 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 109, 'LA'), (110, 119, 'LC'), (120, 126, 'LP'), (131, 143, 'TN'), (169, 188, 'TA'), (189, 198, 'TC'), (199, 205, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Huawei, 802 Bukit Batok West Avenue 5 01-01 Bukit Gombak MRT Station Singapore 223966 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 142, 'LA'), (143, 152, 'LC'), (153, 159, 'LP'), (164, 176, 'TN'), (202, 221, 'TA'), (222, 231, 'TC'), (232, 238, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Dell Technologies, Hitachi Tower 16 Collyer Quay #30-01 Singapore 543846 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 91, 'LN'), (93, 129, 'LA'), (130, 139, 'LC'), (140, 146, 'LP'), (151, 163, 'TN'), (189, 208, 'TA'), (209, 218, 'TC'), (219, 225, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Hitachi, Blk 782, Pasir Ris St. 71 Singapore 805722 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 81, 'LN'), (83, 108, 'LA'), (109, 118, 'LC'), (119, 125, 'LP'), (130, 142, 'TN'), (168, 187, 'TA'), (188, 197, 'TC'), (198, 204, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between IBM, 14 HOUGANG AVENUE 7 01-563 Singapore 919220 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 77, 'LN'), (79, 105, 'LA'), (106, 115, 'LC'), (116, 122, 'LP'), (127, 139, 'TN'), (165, 184, 'TA'), (185, 194, 'TC'), (195, 201, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Sony, HDB-Ang Mo Kio 114 Ang Mo Kio Avenue 4 #01-353 Singapore 466572 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 78, 'LN'), (80, 126, 'LA'), (127, 136, 'LC'), (137, 143, 'LP'), (148, 160, 'TN'), (186, 205, 'TA'), (206, 215, 'TC'), (216, 222, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Panasonic, 185 Haig Road ,01-05 Singapore 114866 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 83, 'LN'), (85, 105, 'LA'), (106, 115, 'LC'), (116, 122, 'LP'), (127, 139, 'TN'), (165, 184, 'TA'), (185, 194, 'TC'), (195, 201, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Intel, 39 Keppel Road BLK 2 #02-04 Tanjong Pagar Distripark 2 Singapore 651574 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 79, 'LN'), (81, 135, 'LA'), (136, 145, 'LC'), (146, 152, 'LP'), (157, 169, 'TN'), (195, 214, 'TA'), (215, 224, 'TC'), (225, 231, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between HP, 3026 Ubi Road 1 #04-160 Singapore 695006 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 76, 'LN'), (78, 101, 'LA'), (102, 111, 'LC'), (112, 118, 'LP'), (123, 135, 'TN'), (161, 180, 'TA'), (181, 190, 'TC'), (191, 197, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Facebook, 126B Dunlop Street Singapore 667212 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 82, 'LN'), (84, 102, 'LA'), (103, 112, 'LC'), (113, 119, 'LP'), (124, 136, 'TN'), (162, 181, 'TA'), (182, 191, 'TC'), (192, 198, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between LG Electronics, 47 Jln Pemimpin #04-08 Singapore 130124 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 88, 'LN'), (90, 112, 'LA'), (113, 122, 'LC'), (123, 129, 'LP'), (134, 146, 'TN'), (172, 191, 'TA'), (192, 201, 'TC'), (202, 208, 'TP')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Lenovo, 157 Yung Loh Road ,05 Singapore 820306 and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(74, 80, 'LN'), (82, 103, 'LA'), (104, 113, 'LC'), (114, 120, 'LP'), (125, 137, 'TN'), (163, 182, 'TA'), (183, 192, 'TC'), (193, 199, 'TP')]}),
              ('This Licence Agreement is made on the hig day of spre mM ber - OS between the Housing and Development Board of HDB Hub, 480 Lorong 6 Toa Payoh Singapore 310480 (“HDB”) and DBS Bank Ltd (UEN No : 196800306E) of 12 Marina Boulevard Singapore 018982', {'entities': [(240, 246, 'TP'), (230, 239, 'TC'), (210, 229, 'TA'), (172, 184, 'TN'), (153, 159, 'LP'), (143, 152, 'LC'), (120, 142, 'LA'), (74, 118, 'LN')]}), ('For and on behalf of DBS Bank Ltd', {'entities': [(21, 33, 'TN')]}), ('HEREAS HDB has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 10, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Aetos Security Management, 10 Shenton Way MAS Building Singapore 079117 (“Aetos") and Bank of Singapore (UEN No : 1233453451A) of 63 Market Street Singapore 876345', {'entities': [(231, 237, 'TP'), (221, 230, 'TC'), (204, 220, 'TA'), (160, 177, 'TN'), (139, 145, 'LP'), (129, 138, 'LC'), (101, 128, 'LA'), (74, 99, 'LN')]}), ('For and on behalf of Bank of Singapore', {'entities': [(21, 38, 'TN')]}), ('HEREAS Aetos has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 12, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Aspial Corporation, 101 kitchener road 02-17 jalan besar plaza Singapore 310480 (“Aspial”) and OCBC Bank (UEN No : 8766543541V) of 65 Chulia Street Singapore 987123', {'entities': [(232, 238, 'TP'), (222, 231, 'TC'), (205, 221, 'TA'), (169, 178, 'TN'), (147, 154, 'LP'), (137, 146, 'LC'), (94, 136, 'LA'), (74, 92, 'LN')]}), ('For and on behalf of OCBC Bank', {'entities': [(21, 30, 'TN')]}), ('HEREAS Aspial has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 13, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Ayam Brand, MDIS Building 190 Changi Road #02-01 Singapore 456345 (“Ayam”) and United Overseas Bank (UEN No : 3452348762S) of 80 Raffles Place Singapore 786409', {'entities': [(227, 233, 'TP'), (217, 226, 'TC'), (200, 216, 'TA'), (153, 173, 'TN'), (133, 139, 'LP'), (123, 132, 'LC'), (86, 122, 'LA'), (74, 84, 'LN')]}), ('For and on behalf of United Overseas Bank', {'entities': [(21, 41, 'TN')]}), ('HEREAS Ayam has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 11, 'LN')]}),
              ('This Licence Agreement is made on the hig day of spre mM ber - OS between Wilmar International, 460 Alexandra Road 22-06 PSA Building Singapore 456384 (“Wilmar”) and Chung Khiaw Bank (UEN No : 835782547F) of 180 Cecil Street Singapore 018982 (“Licensee”).', {'entities': [(235, 241, 'TP'), (225, 234, 'TC'), (208, 224, 'TA'), (166, 182, 'TN'), (153, 159, 'LN'), (144, 150, 'LP'), (134, 143, 'LC'), (96, 133, 'LA'), (74, 94, 'LN')]}), ('For and on behalf of Chung Khiaw Bank', {'entities': [(21, 37, 'TN')]}), ('HEREAS Wilmar has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 13, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Trafigura Group, 52 Greenleaf Drive Singapore 893283 (“Trafigura”) and Hongkong and Shanghai Banking Corporation (UEN No : 847678273R) of 21 Collyer Quay Singapore 982783 (“Licensee”).', {'entities': [(238, 244, 'TP'), (228, 238, 'TC'), (212, 227, 'TA'), (145, 186, 'TN'), (129, 138, 'LN'), (120, 126, 'LP'), (110, 119, 'LC'), (91, 109, 'LA'), (74, 89, 'LN')]}), ('For and on behalf of Hongkong and Shanghai Banking Corporation', {'entities': [(21, 62, 'TN')]}), ('HEREAS Trafigura has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 16, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between AIBI International, 10 Sinaran Drive, 02-02 Square 2 Singapore 847278 (“AIBI”) and Indian Bank (UEN No : 847678273R) of 3 Raffles Place Singapore 823412 (“Licensee”).', {'entities': [(220, 226, 'TP'), (210, 219, 'TC'), (194, 209, 'TA'), (157, 169, 'TN'), (146, 150, 'LN'), (137, 143, 'LP'), (127, 137, 'LC'), (94, 126, 'LA'), (74, 92, 'LN')]}), ('For and on behalf of Indian Bank', {'entities': [(21, 32, 'TN')]}), ('HEREAS AIBI has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 11, 'LN')]}), ('This Licence Agreement is made on the hig day of spre mM ber - OS between Boustead Singapore, 6001 Beach Road 02-37 Golden Mile Tower Singapore 847278 (“Boustead”) and Ping An Bank (UEN No : 847678273R) of 80 Pasir Panjang Road Singapore 987234 (“Licensee”).', {'entities': [(238, 244, 'TP'), (228, 237, 'TC'), (206, 227, 'TA'), (168, 180, 'TN'), (153, 161, 'LN'), (144, 150, 'LP'), (134, 143, 'LC'), (94, 133, 'LA'), (74, 92, 'LN')]}), ('For and on behalf of Indian Bank', {'entities': [(21, 32, 'TN')]}), ('HEREAS Boustead has agreed to grant to the Licensee a Licence to use and occupy the space at Blk 18 Toh Yi Drive #01-113A Singapore 590018 (herein referred to as "Premises") subject to the terms and conditions of this Agreement.', {'entities': [(7, 16, 'LN')]}),
              ('12% per', {'entities': []}),
              ('the execution', {'entities': []}),
              ('the Housing and', {'entities': []}),
              ('Development TC', {'entities': []}),
              ('Act', {'entities': []}),
              ('the Licence may', {'entities': []}),
              ('and in LCI', {'entities': []}),
              ('Be senna ‘hu', {'entities': []}),
              ('4amin £', {'entities': []}),
              ('TEMPORARY OCCUPATION LICENCE FOR', {'entities': []}),
              ('AUTOMATED TELLER MACHINES', {'entities': []}),
              ('This Licence Agreement is made on the hig day of spre mM ber - OS between',
                {'entities': []}),
              ('CASH TENANT_NAME', {'entities': []}),
              ('OFFICE', {'entities': []}),
              ('OCCUPATION LICENCE FOR', {'entities': []}),
              ('TELLER MACHINES', {'entities': []}),
              ('Automated Teller Machine', {'entities': []}),
              ('the Town Councils', {'entities': []}),
              ('Services Tax', {'entities': []}),
              ('Housing and Development Act', {'entities': []}),
              ('(b) To pay and indemnify HDB against GST chargeable in respect of any payment made by the Licensee under the terms of or in connection with this Agreement or in respect of any payment made by HDB where the Licensee has agreed to reimburse HDB for such payment.',
                {'entities': []}),
              ('X Fe', {'entities': []}),
              ('10', {'entities': []}),
              ('11', {'entities': []}),
              ('(d) lf the Licensee fails or neglects to pay the Licence Fees, service and conservancy charges and any other charges due at the time of the termination of the LA Agreement LC',
                {'entities': []}),
              ('to pay an interest at the rate of 12% per annum of the outstanding debt subject to a minimum of $50.00 or such other rate as HDB may from time to time and in its sole discretion, determined.',
                {'entities': []}),
              ('OFFICE', {'entities': []}),
              ('SAM tee PAID', {'entities': []}),
              ('on HHO bok', {'entities': []}),
              ('HOB CASH OFFICE', {'entities': []}),
              ('TEMPORARY OCCUPATION LICENCE FOR', {'entities': []}),
              ('AUTOMATED TELLER MACHINES', {'entities': []}),
              ('This Licence Agreement is made on the hig day of spre mM ber - OS',
                {'entities': []}),
              ('It is hereby agreed as follows:', {'entities': []}),
              ('2 The Licensee hereby also undertakes with HDB as follows: (a) To pay without demand the monthly Licence fee and any other charges pursuant to the provisions of the Town Councils Act 1988 and any amendment thereto, or HDB, as the case may be. All payments, including of Goods and Services Tax (GST), are to be made on the 1st day of each month for that month.',
                {'entities': []}),
              ('(b) To pay and indemnify HDB against GST chargeable in respect of any payment made by the Licensee under the terms of or in connection with this Agreement or in respect of any payment made by HDB where the Licensee has agreed to reimburse HDB for such payment.',
                {'entities': []}),
              ('1988', {'entities': []}),
              ('thereto', {'entities': []}),
              ('or HDB,', {'entities': []}),
              ('as the case may be. All payments, including of Goods and Services Tax (GST), are to be made on the 1st day of each month for that month. (e) To pay on the execution of this Agreement, a sum of $856.00 or such other sum as may be determined by HDB being a deposit which is refundable free of interest by HDB to the Licensee at the expiry or termination of this Agreement. This deposit is to be subject however to all claims by HDB for arrears of Licence Fees and service and conservancy charges, charges for damaged or destroyed fixtures or fittings for repairs to and redecoration of the Premises howsoever occasioned other than by fair wear and tear of the cleaning of the Premises on their vacation by the Licensee and for all other breaches or non observance of covenants contained herein on the part of the Licensee to be performed and observed. The amount and nature of the said replacement of fitting repairs redecoration and cleaning and all claims of HDB are to',
                {'entities': []}),
              ('X Fe', {'entities': []}),
              ('10', {'entities': []}),
              ('11', {'entities': []}),
              ('as the case may be. All payments, including of Goods and Services Tax (GST), are to be made on the 1st day of each month for that month.',
                {'entities': []}),
              ('It is hereby agreed as follows:', {'entities': []}),
              ('1 In consideration of the payment by the Licensee a sum of $800.00 per month (exclusive of Goods and Services Tax) or such other sum as the HDB may notify in writing, HDB hereby authorises the Licensee to use and occupy the Premises for the sole purpose of setting up and maintaining an Automated Teller Machine (ATM) for a period of 25 years commencing from 25/06/2020, subject to renewal, at the discretion of HDB, and upon such terms and conditions as may be imposed by HDB. The licensee is required to give a written request to HDB 2 month prior to the expiry of this TOL to inform of their intention to renew this TOL.',
                {'entities': []}),
              ('2 The Licensee hereby also undertakes with HDB as follows: (a) To pay without demand the monthly Licence fee and any other charges pursuant to the provisions of the Town Councils Act',
                {'entities': []}),
              ('STAMP FEE PAID On_S utfoole M@B_CASH OFFICE', {'entities': []}),
              ('TEMPORARY OCCUPATION LICENCE FOR AUTOMATED TELLER MACHINES',
                {'entities': []}),
              ('It is hereby agreed as follows:', {'entities': []}),
              ('1 In consideration of the payment by the Licensee a sum of $600.00 per month (exclusive of Goods and Services Tax) or such other sum as the HDB may notify in writing, HDB hereby authorises the Licensee to use and occupy the Premises for the sole purpose of setting up and maintaining an Automated Teller Machine (ATM) for a period of 3 years commencing from 01/12/2018, subject to renewal, at the discretion of HDB, and upon such terms and conditions as may be imposed by HDB. The licensee is required to give a written request to HDB 2 month prior to the expiry of this TOL to inform of their intention to renew this TOL.',
                {'entities': []}),
              ('2 The Licensee hereby also undertakes with HDB as follows: (a) To pay without demand the monthly Licence fee and any other charges pursuant to the provisions of the Town Councils Act 1988 and any amendment thereto, or HDB, as the case may be. All payments, including of Goods and Services Tax (GST), are to be made on the 1st day of each month for that month.',
                {'entities': []}),
              ('(b) To pay and indemnify HDB against GST chargeable in respect of any payment made by the Licensee under the terms of or in connection with this Agreement or in respect of any payment made by HDB where the Licensee has agreed to reimburse HDB for such payment.',
                {'entities': []}),
              ('(c) Where applicable, to accept and pay the revised service and conservancy charges as may time to time varied by the Town Council pursuant to the Town Council Act or HDB pursuant to the Housing and Development Act, as the case may be.',
                {'entities': []}),
              ('(d) If the Licensee fails or neglects to pay the Licence Fees, service and conservancy charges and any other charges due at the time of the termination of the Agreement, to pay an interest at the rate of 12% per annum of the outstanding debt subject to a minimum of $50.00 or such other rate as HDB may from time to time and in its sole discretion, determined.',
                {'entities': []}),
              ('(e) To pay on the execution of this Agreement, a sum of $642.00 or such other sum as may be determined by HDB being a deposit which is refundable free of interest by HDB to the Licensee at the expiry or termination of this Agreement. This deposit is to be subject however to all claims by HDB for arrears of Licence Fees and service and conservancy charges, charges for damaged or destroyed fixtures or fittings for repairs to and redecoration of the Premises howsoever occasioned other than by fair wear and tear of the cleaning of the Premises on their vacation by the Licensee and for all other breaches or non observance of covenants contained herein on the part of the Licensee to be performed and observed. The amount and nature of the said replacement of fitting repairs redecoration and cleaning and all claims of HDB are to be decided upon by HDB in its absolute discretion and the Licensee shall not dispute',
                {'entities': []})]


In [222]:
#training data with zero entities

td = []
with open("C:\\Users\\Sivaraman\\Desktop\\Py_projects\\project_ocr\\train_data\\train_mt.txt","r",encoding="utf8") as fil:
    c=0
    for line in fil:
        td.append((line.strip(), {'entities': []}))
print(c)
fil.close()

0


In [227]:
#training on spacy ner

n_iter = 300
model = None
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')   
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [76]:
#training on pre-trained ner

n_iter = 300
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.load('C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_2')   
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [ ]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.2,  
                sgd=optimizer,
                losses=losses)
        print(losses,itn)

In [233]:
#saving model 

output_dir=Path("C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_8")
nlp.to_disk(output_dir)

In [ ]:
#testing on training data

for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#testing on text data

text = new_text.strip()
text = text.rstrip()
text = text.lstrip()
doc = nlp(text)
for ents in doc.ents:
    print(ents.text.strip(),ents.label_)

In [ ]:
#testing saved model on text
        
output_dir = "C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_7"
model = spacy.load(output_dir)
doc =  model(text)
for ent in doc.ents:
    print(ent.text.replace("the space at ",""),ent.label_)

In [42]:
#function to output results as list 

def pred(path,text):
    text = text.strip()
    output_dir=Path(path)
    model = spacy.load(output_dir)
    #model = nlp
    nlp1 = spacy.load("en_core_web_sm")
    doc = nlp1(text)
    doc = model(text.strip())
    LN = []
    LA = []
    LC = []
    LP = []
    TN = []
    TA = []
    TC = []
    TP = []
    for ent in doc.ents:
        if ent.label_ == "LN":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="ORG") and len(LN)==0:
                    LN.append(ent.text)
        elif ent.label_ == "TN":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="ORG") and len(TN)==0:
                    TN.append(ent.text)
        elif ent.label_ == "LC":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="GPE") and len(LC)==0:
                    LC.append(ent.text)
        elif ent.label_ == "TC":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="GPE") and len(TC)==0:
                    TC.append(ent.text)
        elif ent.label_ == "LA":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="CARDINAL") and len(LA)==0:
                    LA.append(ent.text)
        elif ent.label_ == "TA":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="CARDINAL") and len(TA)==0:
                    TA.append(ent.text)
        elif ent.label_ == "LP":
    #         doc1 = nlp1(ent.text.rstrip())
    #         if(doc1.ents):
    #             #if(doc1.ents[0].label_=="CARDINAL") and 
    #             if len(doc1.ents[0].text)==6 and len(LP)==0:
    #                 LP.append(ent.text)
            if len(ent.text) == 6 and len(LP)==0:
                LP.append(ent.text)
        elif ent.label_ == "TP":
    #         doc1 = nlp1(ent.text.rstrip())
    #         print(len(doc1.ents[0].text),len(TP))
    #         if(doc1.ents):
    #             #if(doc1.ents[0].label_=="CARDINAL") and 
    #             if len(doc1.ents[0].text)==6 and len(TP)==0:
    #                 TP.append(ent.text)
            if len(ent.text) == 6 and len(TP)==0:
                TP.append(ent.text)

        print(ent.text.rstrip() , ent.label_)
    #output={"ATenant":"Tenant   "+TN[0],"ALandlord":"Landlord    "+LN[0],"LAddress":"Address   "+LA[0]+LC[0]+LP[0],"TAddress":"Address   "+TA[0]+TC[0]+TP[0]}
    output = []
    output.append(LN)
    output.append(LA)
    output.append(LC)
    output.append(LP)
    output.append(TN)
    output.append(TA)
    output.append(TC)
    output.append(TP)
    return output
output = pred("C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_5",text)

the TESLA LN
345 hougang avenue LA
Singapore LC
310480 BP
DBS Bank Ltd TN
12 Marina Boulevard TA
Singapore TC
018982 TP
the space at
Blk 18 Toh Yi Drive #01-113A BA
Singapore BC
590018 BP


In [133]:
#function to detect suite in given address

def suite(text):
    su = ''
    ind = text.find('#')
    for i in range(ind,len(text)):
        if text[i]!=" ":
            su = su + text[i]
        else:
            break
    return su
#print(suite("sjdfasjhbajs #123-123 akjdadhfbsj"))

#function to calculate date 


def getDate(d):
    date_entry = d
    day, month, year = map(int, date_entry.split('/'))
    return datetime.date(year, month, day)
def addYears(d, years):
    try:
#Return same day of the current year        
        return d.replace(year = d.year + years)
    except ValueError:
#If not same day, it will return other, i.e.  February 29 to March 1 etc.        
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))
def year_calc(da,year):
    string=str(addYears(getDate(da),yrs))
    from datetime import datetime
    oldformat = string
    datetimeobject = datetime.strptime(oldformat,'%Y-%m-%d')
    newformat = datetimeobject.strftime('%d-%m-%Y')
    return newformat
#year_calc("12/01/2020",1)

'12-01-2021'

In [ ]:
#fucntion to append predictions in dictionary

def pred2(paths,text):
    text = text.strip()
    new_text = ""
    for i in range(len(text)):
        if text[i] == '\n' and text[i+1] != '\n':
                new_text = new_text + " "
        else:
            new_text = new_text + op_txt_stp[i]
    mod_path1=Path(paths[0])
    mod_path2=Path(paths[1])
    model = spacy.load(mod_path1)
    model1 = spacy.load(mod_path2)
    #model = nlp
    nlp1 = spacy.load("en_core_web_sm")
    doc1 = nlp1(new_text)
    doc = model(new_text.strip())
    doc2 = model1(new_text.strip())
    LN = []
    LA = []
    LC = []
    LP = []
    TN = []
    TA = []
    TC = []
    TP = []
    BA = []
    BC = []
    SU = []
    BP = []
    YE = []
    DA = []
    for ent in doc.ents:
        if ent.label_ == "LN":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if len(LN)==0 :#(doc1.ents[0].label_=="ORG"):
                    LN.append(ent.text)
        elif ent.label_ == "TN":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if len(TN)==0: #(doc1.ents[0].label_=="ORG") and:
                    TN.append(ent.text)
        elif ent.label_ == "LC":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="GPE") and len(LC)==0:
                    LC.append(ent.text)
        elif ent.label_ == "TC":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="GPE") and len(TC)==0:
                    TC.append(ent.text)
        elif ent.label_ == "LA":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="CARDINAL") and len(LA)==0:
                    LA.append(ent.text)
        elif ent.label_ == "TA":
            doc1 = nlp1(ent.text.rstrip())
            if(doc1.ents):
                if(doc1.ents[0].label_=="CARDINAL") and len(TA)==0:
                    TA.append(ent.text)
        elif ent.label_ == "LP":
    #         doc1 = nlp1(ent.text.rstrip())
    #         if(doc1.ents):
    #             #if(doc1.ents[0].label_=="CARDINAL") and 
    #             if len(doc1.ents[0].text)==6 and len(LP)==0:
    #                 LP.append(ent.text)
            if len(ent.text) == 6 and len(LP)==0:
                LP.append(ent.text)
        elif ent.label_ == "TP":
    #         doc1 = nlp1(ent.text.rstrip())
    #         print(len(doc1.ents[0].text),len(TP))
    #         if(doc1.ents):
    #             #if(doc1.ents[0].label_=="CARDINAL") and 
    #             if len(doc1.ents[0].text)==6 and len(TP)==0:
    #                 TP.append(ent.text)
            if len(ent.text) == 6 and len(TP)==0:
                TP.append(ent.text)
        elif ent.label_ == "BA":
            if len(BA) == 0 and len(SU)==0:
                ad = ent.text.replace("the space at\n","")
                su = suite(ad)
                BA.append(str(ad))
                SU.append(su)
                print(ad)
        elif ent.label_ == "BC" and len(BC)==0:
            BC.append(ent.text.strip())
        elif ent.label_ == "BP" :
            BP.append(ent.text.strip())
        #print(ent.text.rstrip() , ent.label_)
    for ent in doc2.ents:
        if ent.label_ == "DA":
            DA.append(ent.text)
        elif ent.label_ == "YE":
            YE.append(int(ent.text.replace(" years","")))
        #print(ent.text,ent.label_)
    #output={"ATenant":"Tenant   "+TN[0],"ALandlord":"Landlord    "+LN[0],"LAddress":"Address   "+LA[0]+LC[0]+LP[0],"TAddress":"Address   "+TA[0]+TC[0]+TP[0]}
    sqm = text.find("sqm")
    s = ""
    if sqm>0:
        sc = 0
        s = ""
        j = sqm-1
        while sc<2:
            if text[j] == ' ':
                sc=sc + 1
            else:
                s = s + text[j]
            j = j - 1
        s = s[::-1]
    else:
        s = "NA"
    area = s
    date = year_calc(DA[0],YE[0])
    print(date)
    output = {
        "Tenant name": TN[0],
        "Tenant address": TA[0],
        "Tenant city":TC[0],
        "Tenant postal":TP[0],
        "Landlord name":  LN[0],
        "Landlord address":  LA[0],
        "Landlord city": LC[0],
        "Landlord postal": BP[0],
        "Building address": BA[0],
        "Building suite": SU[0],
        "Building city": BC[0],
        "Building postal": BP[1],
        "Date of commencement": DA[0],
        "Years": YE[0],
        "Date of expiry" : date,
        "Area":area
        
    }
    for k in output:
        output[k] = str(output[k]).strip()
#     output = []
#     output.append(LN)
#     output.append(LA)
#     output.append(LC)
#     output.append(LP)
#     output.append(TN)
#     output.append(TA)
#     output.append(TC)
#     output.append(TP)
    return output
paths = ["C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_7","C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_6"]
output = pred2(paths,op_txt_stp)

In [247]:
#output

for k in output:
    print(k,output[k])

Tenant name DBS Bank Ltd.
Tenant address 12 Marina Boulevard
Tenant city Singapore
Tenant postal 018982
Landlord name the Housing and Development Board of HDB Hub
Landlord address 480 Lorong 6 Toa Payoh
Landlord city Singapore
Landlord postal 310480
Building address BLK 1 EVERTON PK #01-27A
Building suite #01-27A
Building city SINGAPORE
Building postal 081001
Date of commencement 01/12/2018
Years 3
Date of expiry 01-12-2019
Area 15.00


In [53]:
#function to fill in pdf file

def fill(path,Dictionary):
    doc = Document("fill2.docx")
    #Dictionary = {"ATenant": "Tenant : DBS bank"}
    for i in Dictionary:
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    for p in cell.paragraphs:
                        if p.text.find(i)>=0:
                            p.text=p.text.replace(i,Dictionary[i])
    #save changed document
    doc.save('./'+path)

    convert(path)
    convert(path, "output.pdf")


In [136]:
#fucntion to fill in csv file

def fill2(path,dic):
    fil = open(path,"w+")
    cs = csv.writer(fil)
    for i in dic:
        cs.writerow([i,dic[i]])
    fil.close()

In [156]:
#predciting and filling in csv

paths = ["C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_5","C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_6"]
output = pred2(paths,text)
fill2("outputs/sample1.csv",output)

In [57]:
#tkinter ui for filling in pdf

root = Tk() 
root.geometry('500x500') 
T = Text(root, height = 10, width = 52) 
l = Label(root, text = "Automated text extraction") 
def open_file(): 
    file = askopenfile(mode ='r', filetypes =[('PDF Files', '*.pdf')]) 
    path = ""
    #print(file.name)
    for i in range(len(file.name)):
        if file.name[i] == '/':
            path = path + "\\"
        else:
            path = path + file.name[i]
    print(path)
    print("Applying OCR....")
    output_text = ocr(path,1)
    print("OCR completed....")
    print("Applying NLP....")
    output = pred("C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_2",output_text)
    print("NLP completed...")
    print("DOC filling...")
    fill("output.docx",output)
    print("Process completed...")
    
    
#     if file is not None: 
#         content = file.read() 
#         print(content) 
l.pack()
btn = Button(root, text ='Open', command = lambda:open_file()) 
btn.pack(side = TOP, pady = 10) 
l.pack()
  
mainloop() 

C:\Users\Sivaraman\Desktop\Py_projects\project_ocr\extract.pdf
Applying OCR....
OCR completed....
Applying NLP....
NLP completed...
DOC filling...


Process completed...


In [158]:
#tkinter ui for filling in csv

root = Tk() 
root.geometry('500x500') 
T = Text(root, height = 10, width = 52) 
l = Label(root, text = "Automated text extraction") 
def open_file(): 
    file = askopenfile(mode ='r', filetypes =[('PDF Files', '*.pdf')]) 
    path = ""
    #print(file.name)
    for i in range(len(file.name)):
        if file.name[i] == '/':
            path = path + "\\"
        else:
            path = path + file.name[i]
    print(path)
    print("Applying OCR....")
    output_text = ocr(path,1)
    print("OCR completed....")
    print("Applying NLP....")
    paths = ["C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_7","C:\\Users\\Sivaraman\\Desktop\\Py_projects\\spacy_models\\model_6"]
    output = pred2(paths,output_text)
    print("NLP completed...")
    print("CSV filling...")
    fill2("./outputs/sample1.csv",output)
    print("Process completed...")
    
    
#     if file is not None: 
#         content = file.read() 
#         print(content) 
l.pack()
btn = Button(root, text ='Open', command = lambda:open_file()) 
btn.pack(side = TOP, pady = 10) 
l.pack()
  
mainloop() 

C:\Users\Sivaraman\Desktop\Py_projects\project_ocr\extract_2.pdf
Applying OCR....
OCR completed....
Applying NLP....
Singapore BC
310480 BP
DBS Bank Ltd. TN
12 Marina Boulevard TA
Singapore TC
018982 BP
notdisput
01-12-2019


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Sivaraman\miniconda3\envs\tensorflow\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-158-38f96002c5b7>", line 31, in <lambda>
    btn = Button(root, text ='Open', command = lambda:open_file())
  File "<ipython-input-158-38f96002c5b7>", line 20, in open_file
    output = pred2(paths,output_text)
  File "<ipython-input-154-984627fa52a7>", line 113, in pred2
    "Tenant postal":TP[0],
IndexError: list index out of range
